In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Installing packages - **DONT TOUCH IT**

In [9]:
!git clone https://github.com/pallets/flask.git

fatal: destination path 'flask' already exists and is not an empty directory.


In [ ]:
!pip install nx-arangodb

In [11]:
!nvidia-smi
!nvcc --version

zsh:1: command not found: nvidia-smi
zsh:1: command not found: nvcc


In [12]:
!pip install nx-cugraph-cu12 --extra-index-url https://pypi.nvidia.com # Requires CUDA-capable GPU

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
INFO: pip is looking at multiple versions of nx-cugraph-cu12 to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of nx-cugraph-cu12 to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install nx-cugraph-cu12==23.10.0, nx-cugraph-cu12==23.12.0, nx-cugraph-cu12==24.10.0, nx-cugraph-cu12==24.12.0, nx-cugraph-cu12==24.2.0, nx-cugraph-cu12==24.4.0, nx-cugraph-cu12==24.6.0, nx-cugraph-cu12==24.6.1, nx-cugraph-cu12==24.8.0 and nx-cugraph-cu12==25.2.0 because these package versions have conflicting dependencies.

The conflict is caused by:
    nx-cugraph-cu12 25.2.0 depends on cupy-cuda12x>=12.0.0
    nx-cugraph-cu12 24.12.0 depends on cupy-cuda12x>=12.0.0
    nx-cugraph-cu12 24.10.0 depends on cupy-cuda12x>=12.0.0
    nx-cugraph-cu12 24.8.0 depends on cupy-cuda12x>=12.0.0
    nx-cugraph-

OSError: [Errno 5] Input/output error

In [ ]:
!pip install --upgrade langchain langchain-community langchain-openai langgraph langchain_mistralai

In [14]:
!pip install networkx==3.4

In [15]:
%pip install tree-sitter

Note: you may need to restart the kernel to use updated packages.


In [16]:
!git clone https://github.com/tree-sitter/tree-sitter-cpp

fatal: destination path 'tree-sitter-cpp' already exists and is not an empty directory.


In [11]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import os
import ast
from typing import Dict, Set, List, Tuple, Optional,Any
import json
from arango import ArangoClient
import nx_arangodb as nxadb
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool
from langchain_mistralai import ChatMistralAI
import re

[19:32:44 +0530] [INFO]: NetworkX-cuGraph is unavailable: No module named 'cupy'.


### Building graph

In [ ]:
# import os
# import ast
# import networkx as nx
# from typing import Dict, Set, List, Tuple, Optional
# import json
# from arango import ArangoClient
# import re

# class CodebaseVisualizer:
#     def __init__(self, root_dir: str):
#         self.root_dir = root_dir
#         self.graph = nx.DiGraph()
#         self.file_contents: Dict[str, str] = {}
#         self.import_relations: Dict[str, List[Tuple[str, int]]] = {}  # file -> [(module, line_no)]
#         self.module_symbols: Dict[str, Dict[str, Dict[str, any]]] = {}  # file -> {symbol -> {type, line_no, context}}
#         self.symbol_references: Dict[str, List[Tuple[str, int]]] = {}  # symbol -> [(file, line_no)]
#         self.file_index: Dict[str, int] = {}  # Maps files to indices
#         self.current_index = 0
#         self.directories: Set[str] = set()
#         # Add a new index for all symbols to quickly locate them
#         self.symbol_index: Dict[str, List[Dict]] = {}  # symbol -> [{file, type, line_no, context}]

#     def _get_next_index(self) -> int:
#         """Get next available index for file indexing."""
#         self.current_index += 1
#         return self.current_index

#     def _chunk_code(self, code: str, lines_per_chunk: int = 20) -> List[Dict]:
#         """
#         Chunk the given code into snippets.
#         Returns a list of dictionaries with 'code_snippet', 'start_line', and 'end_line'.
#         """
#         lines = code.splitlines()
#         chunks = []
#         for i in range(0, len(lines), lines_per_chunk):
#             chunk_lines = lines[i:i + lines_per_chunk]
#             chunk = {
#                 'code_snippet': '\n'.join(chunk_lines),
#                 'start_line': i + 1,
#                 'end_line': i + len(chunk_lines)
#             }
#             chunks.append(chunk)
#         return chunks

#     def _get_context_around_line(self, file_path: str, line_no: int, context_lines: int = 3) -> str:
#         """Extract context around a specific line in a file."""
#         if file_path not in self.file_contents:
#             return ""
        
#         lines = self.file_contents[file_path].splitlines()
#         start = max(0, line_no - context_lines - 1)
#         end = min(len(lines), line_no + context_lines)
        
#         context = "\n".join(lines[start:end])
#         return context

#     def parse_files(self) -> None:
#         """Parse all Python files in the directory and build relationships."""
#         # First pass: Index all files and create directory nodes
#         for root, dirs, files in os.walk(self.root_dir):
#             # Add directory node
#             rel_dir = os.path.relpath(root, self.root_dir)
#             if rel_dir != '.':
#                 self.directories.add(rel_dir)
#                 self.graph.add_node(rel_dir, type='directory')

#             # Index Python files
#             for file in files:
#                 if file.endswith('.py'):
#                     file_path = os.path.join(root, file)
#                     rel_path = os.path.relpath(file_path, self.root_dir)
#                     self.file_index[rel_path] = self._get_next_index()
                    
#                     try:
#                         with open(file_path, 'r', encoding='utf-8') as f:
#                             content = f.read()
#                             self.file_contents[rel_path] = content
#                             self._analyze_file(rel_path, content)
#                     except Exception as e:
#                         print(f"Error parsing {file_path}: {e}")
        
#         # Second pass: Find symbol references across files
#         self._find_symbol_references()
        
#         # Build the symbol index after all analyses
#         self._build_symbol_index()

#     def _analyze_file(self, file_path: str, content: str) -> None:
#         """Analyze a single file for imports and symbols with line numbers and context."""
#         try:
#             tree = ast.parse(content)
#             imports = []
#             symbols = {}

#             for node in ast.walk(tree):
#                 # Track imports
#                 if isinstance(node, (ast.Import, ast.ImportFrom)):
#                     if isinstance(node, ast.Import):
#                         for name in node.names:
#                             imports.append((name.name, node.lineno))
#                     else:  # ImportFrom
#                         module = node.module if node.module else ''
#                         for name in node.names:
#                             imports.append((f"{module}.{name.name}" if module else name.name, node.lineno))

#                 # Track defined symbols with line numbers and context
#                 elif isinstance(node, (ast.FunctionDef, ast.ClassDef, ast.Assign)):
#                     if isinstance(node, (ast.FunctionDef, ast.ClassDef)):
#                         symbol_name = node.name
#                         symbol_type = 'class' if isinstance(node, ast.ClassDef) else 'function'
#                         line_no = node.lineno
#                         context = self._extract_node_source(content, node)
                        
#                         symbols[symbol_name] = {
#                             'type': symbol_type,
#                             'line_no': line_no,
#                             'context': context,
#                             'docstring': ast.get_docstring(node)
#                         }
#                     elif isinstance(node, ast.Assign):
#                         # Handle variable assignments
#                         for target in node.targets:
#                             if isinstance(target, ast.Name):
#                                 symbol_name = target.id
#                                 line_no = node.lineno
#                                 context = self._extract_node_source(content, node)
                                
#                                 symbols[symbol_name] = {
#                                     'type': 'variable',
#                                     'line_no': line_no,
#                                     'context': context
#                                 }

#             self.import_relations[file_path] = imports
#             self.module_symbols[file_path] = symbols

#         except Exception as e:
#             print(f"Error analyzing {file_path}: {e}")

#     def _extract_node_source(self, source: str, node) -> str:
#         """Extract the source code for an AST node."""
#         try:
#             lines = source.splitlines()
#             if hasattr(node, 'lineno') and hasattr(node, 'end_lineno'):
#                 start = node.lineno - 1
#                 end = getattr(node, 'end_lineno', start + 1)
#                 return '\n'.join(lines[start:end])
#             return ""
#         except Exception:
#             return ""

#     def _find_symbol_references(self) -> None:
#         """Find references to symbols across all files."""
#         for file_path, content in self.file_contents.items():
#             try:
#                 tree = ast.parse(content)
#                 self._process_file_for_references(file_path, tree, content)
#             except Exception as e:
#                 print(f"Error finding references in {file_path}: {e}")

#     def _process_file_for_references(self, file_path: str, tree, source: str) -> None:
#         """Process a file's AST to find references to symbols."""
#         for node in ast.walk(tree):
#             # Find variable references
#             if isinstance(node, ast.Name) and isinstance(node.ctx, ast.Load):
#                 symbol_name = node.id
#                 line_no = node.lineno
                
#                 # Track reference with context
#                 if symbol_name not in self.symbol_references:
#                     self.symbol_references[symbol_name] = []
                
#                 context = self._get_context_around_line(file_path, line_no)
#                 self.symbol_references[symbol_name].append((file_path, line_no, context))
            
#             # Find attribute references (e.g., obj.method())
#             elif isinstance(node, ast.Attribute) and isinstance(node.ctx, ast.Load):
#                 attr_name = node.attr
#                 line_no = node.lineno
                
#                 if attr_name not in self.symbol_references:
#                     self.symbol_references[attr_name] = []
                
#                 context = self._get_context_around_line(file_path, line_no)
#                 self.symbol_references[attr_name].append((file_path, line_no, context))

#     def _build_symbol_index(self) -> None:
#         """Build a comprehensive index of all symbols and where they're defined/used."""
#         # Initialize the symbol index
#         self.symbol_index = {}
        
#         # First, add all symbol definitions
#         for file_path, symbols in self.module_symbols.items():
#             for symbol_name, details in symbols.items():
#                 if symbol_name not in self.symbol_index:
#                     self.symbol_index[symbol_name] = []
                
#                 self.symbol_index[symbol_name].append({
#                     'file': file_path,
#                     'type': 'definition',
#                     'symbol_type': details['type'],
#                     'line_no': details['line_no'],
#                     'context': details.get('context', ''),
#                     'docstring': details.get('docstring', '')
#                 })
        
#         # Then, add all references
#         for symbol_name, references in self.symbol_references.items():
#             if symbol_name not in self.symbol_index:
#                 self.symbol_index[symbol_name] = []
            
#             for file_path, line_no, context in references:
#                 # Avoid duplicating references if they're already in definitions
#                 if not any(ref['file'] == file_path and ref['line_no'] == line_no and ref['type'] == 'definition' 
#                           for ref in self.symbol_index.get(symbol_name, [])):
#                     self.symbol_index[symbol_name].append({
#                         'file': file_path,
#                         'type': 'reference',
#                         'line_no': line_no,
#                         'context': context
#                     })

#     def build_graph(self) -> nx.DiGraph:
#         """Build the NetworkX graph with enhanced node and edge information."""
#         # Start with a directed graph for clarity in relationships
#         dot_graph = nx.DiGraph()
        
#         # Add nodes for all files with indices and code snippet nodes
#         for file_path, file_idx in self.file_index.items():
#             dot_graph.add_node(file_path, 
#                                type='file',
#                                file_index=file_idx,
#                                directory=os.path.dirname(file_path))
            
#             # Create snippet nodes for the entire file
#             if file_path in self.file_contents:
#                 chunks = self._chunk_code(self.file_contents[file_path])
#                 for idx, chunk_info in enumerate(chunks):
#                     snippet_node = f"{file_path}::snippet::{idx}"
#                     dot_graph.add_node(snippet_node,
#                                        type='snippet',
#                                        code_snippet=chunk_info['code_snippet'],
#                                        start_line=chunk_info['start_line'],
#                                        end_line=chunk_info['end_line'])
#                     # Connect file node to snippet node
#                     dot_graph.add_edge(file_path, snippet_node, 
#                                        edge_type='contains_snippet',
#                                        start_line=chunk_info['start_line'],
#                                        end_line=chunk_info['end_line'])

#             # Add nodes for symbols in this file
#             for symbol, details in self.module_symbols.get(file_path, {}).items():
#                 symbol_node = f"{file_path}::{symbol}"
#                 dot_graph.add_node(symbol_node, 
#                                    type='symbol',
#                                    symbol_type=details['type'],
#                                    line_number=details['line_no'],
#                                    context=details.get('context', ''),
#                                    docstring=details.get('docstring', ''))
#                 dot_graph.add_edge(file_path, symbol_node, 
#                                    edge_type='defines',
#                                    line_number=details['line_no'])

#         # Add directory nodes
#         for directory in self.directories:
#             dot_graph.add_node(directory, type='directory')

#         # Add edges for imports with line numbers
#         for file_path, imports in self.import_relations.items():
#             for imp, line_no in imports:
#                 # Look for matching files or symbols
#                 for target_file, symbols in self.module_symbols.items():
#                     if imp in symbols:
#                         dot_graph.add_edge(file_path, 
#                                            f"{target_file}::{imp}",
#                                            edge_type='import',
#                                            line_number=line_no)
#                     elif target_file.replace('.py', '').endswith(imp):
#                         dot_graph.add_edge(file_path, 
#                                            target_file,
#                                            edge_type='import',
#                                            line_number=line_no)
        
#         # Add edges for symbol references with line numbers and context
#         for symbol, references in self.symbol_references.items():
#             for file_path, symbols in self.module_symbols.items():
#                 if symbol in symbols:
#                     symbol_node = f"{file_path}::{symbol}"
                    
#                     # Connect symbol to all its references
#                     for ref_file, ref_line, context in references:
#                         if ref_file != file_path:  # Only add cross-file references
#                             dot_graph.add_edge(ref_file, 
#                                               symbol_node,
#                                               edge_type='references',
#                                               line_number=ref_line,
#                                               context=context)
        
#         # Save the built graph in self.graph for later export
#         self.graph = dot_graph
#         return dot_graph

#     def find_symbol_usages(self, symbol_name: str) -> List[Dict]:
#         """Find all usages of a symbol in the codebase with context."""
#         # Use the symbol index for fast lookup
#         if symbol_name in self.symbol_index:
#             return self.symbol_index[symbol_name]
#         return []

#     def find_symbol_by_partial_name(self, partial_name: str) -> Dict[str, List[Dict]]:
#         """Find symbols that match the partial name (case-insensitive)."""
#         results = {}
#         partial_name_lower = partial_name.lower()
        
#         for symbol_name, occurrences in self.symbol_index.items():
#             if partial_name_lower in symbol_name.lower():
#                 results[symbol_name] = occurrences
        
#         return results

#     def get_symbol_definitions(self, symbol_name: str) -> List[Dict]:
#         """Get all definitions of a symbol across the codebase."""
#         if symbol_name in self.symbol_index:
#             return [item for item in self.symbol_index[symbol_name] if item['type'] == 'definition']
#         return []

#     def get_symbol_references(self, symbol_name: str) -> List[Dict]:
#         """Get all references to a symbol across the codebase."""
#         if symbol_name in self.symbol_index:
#             return [item for item in self.symbol_index[symbol_name] if item['type'] == 'reference']
#         return []

#     def infer_symbol_location(self, symbol_query: str) -> Dict:
#         """
#         Infer the location and details of a symbol based on a query.
#         This function makes it easier for LLMs to find the right symbol.
#         """
#         # First, try exact match
#         if symbol_query in self.symbol_index:
#             definitions = self.get_symbol_definitions(symbol_query)
#             if definitions:
#                 return {
#                     'found': True,
#                     'exact_match': True,
#                     'symbol_name': symbol_query,
#                     'definitions': definitions,
#                     'references': self.get_symbol_references(symbol_query)
#                 }
        
#         # Try partial match
#         partial_matches = self.find_symbol_by_partial_name(symbol_query)
#         if partial_matches:
#             # Sort matches by relevance (length of partial match compared to full name)
#             sorted_matches = sorted(
#                 partial_matches.items(), 
#                 key=lambda x: (abs(len(x[0]) - len(symbol_query)), x[0])
#             )
            
#             # Get the most relevant matches
#             relevant_symbols = {}
#             for symbol_name, occurrences in sorted_matches[:5]:  # Take top 5 matches
#                 definitions = [item for item in occurrences if item['type'] == 'definition']
#                 references = [item for item in occurrences if item['type'] == 'reference']
                
#                 if definitions:  # Only include symbols with definitions
#                     relevant_symbols[symbol_name] = {
#                         'definitions': definitions,
#                         'references': references
#                     }
            
#             if relevant_symbols:
#                 return {
#                     'found': True,
#                     'exact_match': False,
#                     'partial_matches': relevant_symbols
#                 }
        
#         # No matches found
#         return {
#             'found': False,
#             'message': f"No symbols matching '{symbol_query}' found in the codebase."
#         }

#     def export_file_index(self, output_path: str = 'file_index.txt') -> None:
#         """Export the file index mapping to a text file."""
#         with open(output_path, 'w') as f:
#             # Write header
#             f.write("File Index Mapping\n")
#             f.write("=================\n\n")
            
#             # Sort by index for better readability
#             sorted_items = sorted(self.file_index.items(), key=lambda x: x[1])
            
#             # Write each file and its index
#             for file_path, index in sorted_items:
#                 f.write(f"{index}: {file_path}\n")
                
#                 # If there are symbols in this file, list them with line numbers
#                 if file_path in self.module_symbols:
#                     f.write("  Symbols:\n")
#                     for symbol, details in self.module_symbols[file_path].items():
#                         symbol_type = details['type']
#                         line_no = details['line_no']
#                         f.write(f"    - {symbol} ({symbol_type}) at line {line_no}\n")
#                     f.write("\n")

#     def export_symbol_index(self, output_path: str = 'symbol_index.json') -> None:
#         """Export the symbol index to a JSON file for quick lookups."""
#         with open(output_path, 'w') as f:
#             json.dump(self.symbol_index, f, indent=2)

#     def _analyze_symbol_purpose(self, symbol_name, usages):
#         """Analyze the purpose of a symbol based on its usage patterns."""
#         # Get the definition if available
#         definitions = [u for u in usages if u['type'] == 'definition']
#         references = [u for u in usages if u['type'] == 'reference']
        
#         purpose = ""
        
#         # Check if we have a definition with docstring
#         if definitions and definitions[0].get('docstring'):
#             purpose += f"{definitions[0]['docstring']}\n\n"
        
#         # If it's a function, try to infer what it does from usage
#         if definitions and definitions[0].get('symbol_type') == 'function':
#             # Collect contexts where it's used
#             contexts = [ref.get('context', '') for ref in references if ref.get('context')]
            
#             # Analyze contexts for patterns
#             if contexts:
#                 common_patterns = self._find_common_usage_patterns(contexts, symbol_name)
                
#                 purpose += "Based on usage patterns, this function appears to:\n"
                
#                 if any("assert" in ctx for ctx in contexts):
#                     purpose += "- Be used in test assertions to verify behavior\n"
                
#                 if any("if" in ctx and symbol_name in ctx for ctx in contexts):
#                     purpose += "- Be used as a condition in control flow statements\n"
                
#                 if common_patterns:
#                     for pattern in common_patterns[:3]:  # Top 3 patterns
#                         purpose += f"- {pattern}\n"
        
#         # If we couldn't infer much, provide a generic description
#         if not purpose or len(purpose.strip()) < 10:
#             if definitions and definitions[0].get('symbol_type') == 'function':
#                 context = definitions[0].get('context', '')
                
#                 # Look for parameters to understand what it takes
#                 params = self._extract_function_params(context)
                
#                 purpose += f"This function appears to check or validate something"
#                 if params:
#                     purpose += f" related to {', '.join(params)}"
#                 purpose += ".\n"
        
#         return purpose

#     def _find_common_usage_patterns(self, contexts, symbol_name):
#         """Find common patterns in the usage contexts."""
#         patterns = []
        
#         # Check if it's used with certain objects/methods frequently
#         if any(f".{symbol_name}" in ctx for ctx in contexts):
#             patterns.append("Be a method called on objects")
        
#         # Check if it's used for configuration or setup
#         if any("config" in ctx.lower() or "setup" in ctx.lower() for ctx in contexts):
#             patterns.append("Be involved in configuration or setup")
        
#         # Check if it's used for logging
#         if any("log" in ctx.lower() for ctx in contexts):
#             patterns.append("Be related to logging functionality")
        
#         # Check if it's used in exception handling
#         if any("except" in ctx.lower() or "try" in ctx.lower() for ctx in contexts):
#             patterns.append("Be involved in exception handling")
        
#         return patterns

#     def _extract_function_params(self, context):
#         """Extract parameter names from a function definition."""
#         params = []
#         if context:
#             # Simple regex-based extraction
#             match = re.search(r'def\s+\w+\s*\((.*?)\)', context, re.DOTALL)
#             if match:
#                 param_string = match.group(1)
#                 # Split by comma and clean up
#                 raw_params = [p.strip() for p in param_string.split(',')]
#                 # Extract just the parameter name (before any : or =)
#                 params = [re.split(r'[=:]', p)[0].strip() for p in raw_params if p]
#                 # Remove self if it's there
#                 if params and params[0] == 'self':
#                     params = params[1:]
#         return params

#     def _general_codebase_analysis(self, query):
#         """Provide a general analysis based on the query."""
#         response = f"# Analysis for Query: {query}\n\n"
        
#         # Check if the query is asking about structure
#         if any(term in query.lower() for term in ['structure', 'organization', 'layout']):
#             response += "## Codebase Structure\n\n"
#             # Count files by directory
#             files_by_dir = {}
#             for file_path in self.file_index:
#                 directory = os.path.dirname(file_path)
#                 if directory not in files_by_dir:
#                     files_by_dir[directory] = []
#                 files_by_dir[directory].append(file_path)
            
#             response += f"The codebase contains {len(self.file_index)} Python files across {len(files_by_dir)} directories.\n\n"
            
#             # Show top-level directories
#             response += "Main directories:\n"
#             for directory, files in sorted(files_by_dir.items(), key=lambda x: len(x[1]), reverse=True)[:10]:
#                 dir_name = directory if directory else '(root)'
#                 response += f"- {dir_name}: {len(files)} files\n"
        
#         # Check if the query is asking about specific functionality
#         functionality_terms = ['handle', 'process', 'create', 'generate', 'calculate']
#         for term in functionality_terms:
#             if term in query.lower():
#                 # Search for functions with this term
#                 matching_functions = []
#                 for file_path, symbols in self.module_symbols.items():
#                     for symbol, details in symbols.items():
#                         if details['type'] == 'function' and term in symbol.lower():
#                             matching_functions.append((file_path, symbol, details))
                
#                 if matching_functions:
#                     response += f"\n## Functions Related to '{term}'\n\n"
#                     for file_path, symbol, details in matching_functions[:5]:  # Show top 5
#                         response += f"- `{symbol}` in {file_path}:{details['line_no']}\n"
#                     if len(matching_functions) > 5:
#                         response += f"... and {len(matching_functions) - 5} more functions\n"
        
#         return response

#     def export_graph_json(self, output_path: str = 'codebase_graph.json') -> None:
#         """Export the graph structure to JSON."""
#         graph_data = {
#             'nodes': [
#                 {
#                     'id': node,
#                     'type': data['type'],
#                     'file_index': data.get('file_index'),
#                     'directory': data.get('directory'),
#                     'symbol_type': data.get('symbol_type'),
#                     'line_number': data.get('line_number'),
#                     'code_snippet': data.get('code_snippet'),
#                     'start_line': data.get('start_line'),
#                     'end_line': data.get('end_line'),
#                     'context': data.get('context'),
#                     'docstring': data.get('docstring')
#                 } 
#                 for node, data in self.graph.nodes(data=True)
#             ],
#             'links': [
#                 {
#                     'source': source,
#                     'target': target,
#                     'type': data.get('edge_type'),
#                     'line_number': data.get('line_number'),
#                     'start_line': data.get('start_line'),
#                     'end_line': data.get('end_line'),
#                     'context': data.get('context')
#                 } 
#                 for source, target, data in self.graph.edges(data=True)
#             ]
#         }
        
#         with open(output_path, 'w') as f:
#             json.dump(graph_data, f, indent=2)
            
#     def export_inference_data(self, output_path: str = 'symbol_inference.json') -> None:
#         """
#         Export inference-friendly data about all symbols in the codebase.
#         This creates a dedicated lookup file optimized for LLMs to find symbols.
#         """
#         inference_data = {
#             'symbols': {},
#             'metadata': {
#                 'total_symbols': len(self.symbol_index),
#                 'total_files': len(self.file_index),
#                 'total_directories': len(self.directories)
#             }
#         }
        
#         # Process each symbol with its definitions and references
#         for symbol_name, occurrences in self.symbol_index.items():
#             definitions = [item for item in occurrences if item['type'] == 'definition']
#             references = [item for item in occurrences if item['type'] == 'reference']
            
#             if definitions:  # Only include symbols with at least one definition
#                 # Create a symbol entry with all relevant information
#                 symbol_entry = {
#                     'definition_files': list(set(d['file'] for d in definitions)),
#                     'definition_count': len(definitions),
#                     'reference_count': len(references),
#                     'definitions': definitions,
#                     # Include just summarized references to keep file size manageable
#                     'reference_summary': {
#                         f['file']: len([r for r in references if r['file'] == f['file']])
#                         for f in sorted(set(({'file': r['file']} for r in references)), key=lambda x: x['file'])
#                     },
#                     'symbol_types': list(set(d.get('symbol_type', 'unknown') for d in definitions)),
#                     'purpose': self._analyze_symbol_purpose(symbol_name, occurrences) if definitions else ""
#                 }
                
#                 inference_data['symbols'][symbol_name] = symbol_entry
        
#         with open(output_path, 'w') as f:
#             json.dump(inference_data, f, indent=2)

#     def export_to_arango(self,
#                          db_name: str = 'codebase',
#                          username: str = 'root',
#                          password: str = 'passwd',
#                          host: str = 'http://localhost:8529') -> None:
#         """Export the graph into ArangoDB."""
#         client = ArangoClient(hosts=host)
#         db = client.db(username=username, password=password, verify=True)

#         # Delete the existing graph and its collections if they exist.
#         graph_name = "FlaskRepv1"
        
#         # Import networkx_to_arangodb if it's available
#         try:
#             import networkx_to_arangodb as nxadb # type: ignore
#             G_adb = nxadb.Graph(
#                 name=graph_name,
#                 db=db,
#                 incoming_graph_data=self.graph,
#                 write_batch_size=50000,
#                 overwrite_graph=True
#             )
            
#             self.G_adb = G_adb
#             print("Graph successfully exported to ArangoDB.")
#             return G_adb
#         except ImportError:
#             print("networkx_to_arangodb module not found. Please install it to export to ArangoDB.")
#             return None

In [ ]:
import os
import ast
import networkx as nx
from typing import Dict, Set, List, Tuple, Optional, Union
import json
from arango import ArangoClient
import re
import glob

class CodebaseVisualizer:
    def __init__(self, root_dir: str, supported_languages=None):
        self.root_dir = root_dir
        self.graph = nx.DiGraph()
        self.file_contents: Dict[str, str] = {}
        self.import_relations: Dict[str, List[Tuple[str, int]]] = {}  # file -> [(module, line_no)]
        self.module_symbols: Dict[str, Dict[str, Dict[str, any]]] = {}  # file -> {symbol -> {type, line_no, context}}
        self.symbol_references: Dict[str, List[Tuple[str, int, str]]] = {}  # symbol -> [(file, line_no, context)]
        self.file_index: Dict[str, int] = {}  # Maps files to indices
        self.current_index = 0
        self.directories: Set[str] = set()
        # Add a new index for all symbols to quickly locate them
        self.symbol_index: Dict[str, List[Dict]] = {}  # symbol -> [{file, type, line_no, context}]
        
        # Define supported languages
        self.supported_languages = supported_languages or ["python", "cpp", "java", "go"]
        
        # Language file extensions mapping
        self.language_extensions = {
            "python": [".py"],
            "cpp": [".c", ".cpp", ".h", ".hpp", ".cc", ".cxx", ".hxx"],
            "java": [".java"],
            "go": [".go"]
        }

    def _get_next_index(self) -> int:
        """Get next available index for file indexing."""
        self.current_index += 1
        return self.current_index

    def _chunk_code(self, code: str, lines_per_chunk: int = 20) -> List[Dict]:
        """
        Chunk the given code into snippets.
        Returns a list of dictionaries with 'code_snippet', 'start_line', and 'end_line'.
        """
        lines = code.splitlines()
        chunks = []
        for i in range(0, len(lines), lines_per_chunk):
            chunk_lines = lines[i:i + lines_per_chunk]
            chunk = {
                'code_snippet': '\n'.join(chunk_lines),
                'start_line': i + 1,
                'end_line': i + len(chunk_lines)
            }
            chunks.append(chunk)
        return chunks

    def _get_context_around_line(self, file_path: str, line_no: int, context_lines: int = 3) -> str:
        """Extract context around a specific line in a file."""
        if file_path not in self.file_contents:
            return ""
        
        lines = self.file_contents[file_path].splitlines()
        start = max(0, line_no - context_lines - 1)
        end = min(len(lines), line_no + context_lines)
        
        context = "\n".join(lines[start:end])
        return context

    def _detect_language(self, file_path: str) -> str:
        """Detect the programming language of a file based on its extension."""
        _, ext = os.path.splitext(file_path)
        ext = ext.lower()
        
        for language, extensions in self.language_extensions.items():
            if ext in extensions:
                return language
                
        return "unknown"

    def parse_files(self) -> None:
        """Parse all files in the directory and build relationships."""
        # First pass: Index all files and create directory nodes
        for root, dirs, files in os.walk(self.root_dir):
            # Add directory node
            rel_dir = os.path.relpath(root, self.root_dir)
            if rel_dir != '.':
                self.directories.add(rel_dir)
                self.graph.add_node(rel_dir, type='directory')
                
                # Add edge from parent directory to this directory (if not root)
                parent_dir = os.path.dirname(rel_dir)
                if parent_dir and parent_dir != '.':
                    self.graph.add_edge(parent_dir, rel_dir, edge_type='contains_directory')

            # Index files of supported languages
            for file in files:
                file_path = os.path.join(root, file)
                rel_path = os.path.relpath(file_path, self.root_dir)
                file_language = self._detect_language(file_path)
                
                if file_language in self.supported_languages:
                    self.file_index[rel_path] = self._get_next_index()
                    
                    # Add node for this file
                    self.graph.add_node(rel_path, type='file', file_index=self.file_index[rel_path], language=file_language)
                    
                    # Connect file to its directory
                    if rel_dir != '.':
                        self.graph.add_edge(rel_dir, rel_path, edge_type='contains_file')
                    
                    try:
                        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                            content = f.read()
                            self.file_contents[rel_path] = content
                            self._analyze_file(rel_path, content, file_language)
                    except Exception as e:
                        print(f"Error parsing {file_path}: {e}")
        
        # Second pass: Find symbol references across files
        for file_path, content in self.file_contents.items():
            file_language = self._detect_language(file_path)
            self._find_references_in_file(file_path, content, file_language)
        
        # Build the symbol index after all analyses
        self._build_symbol_index()

    def _analyze_file(self, file_path: str, content: str, language: str) -> None:
        """Analyze a file for imports and symbols with line numbers and context."""
        if language == "python":
            self._analyze_python_file(file_path, content)
        elif language == "cpp":
            self._analyze_cpp_file(file_path, content)
        elif language == "java":
            self._analyze_java_file(file_path, content)
        elif language == "go":
            self._analyze_go_file(file_path, content)

    def _analyze_python_file(self, file_path: str, content: str) -> None:
        """Analyze a Python file for imports and symbols."""
        try:
            tree = ast.parse(content)
            imports = []
            symbols = {}

            for node in ast.walk(tree):
                # Track imports
                if isinstance(node, (ast.Import, ast.ImportFrom)):
                    if isinstance(node, ast.Import):
                        for name in node.names:
                            imports.append((name.name, node.lineno))
                    else:  # ImportFrom
                        module = node.module if node.module else ''
                        for name in node.names:
                            imports.append((f"{module}.{name.name}" if module else name.name, node.lineno))

                # Track defined symbols with line numbers and context
                elif isinstance(node, (ast.FunctionDef, ast.ClassDef, ast.Assign)):
                    if isinstance(node, (ast.FunctionDef, ast.ClassDef)):
                        symbol_name = node.name
                        symbol_type = 'class' if isinstance(node, ast.ClassDef) else 'function'
                        line_no = node.lineno
                        context = self._extract_python_node_source(content, node)
                        
                        symbols[symbol_name] = {
                            'type': symbol_type,
                            'line_no': line_no,
                            'context': context,
                            'docstring': ast.get_docstring(node)
                        }
                    elif isinstance(node, ast.Assign):
                        # Handle variable assignments
                        for target in node.targets:
                            if isinstance(target, ast.Name):
                                symbol_name = target.id
                                line_no = node.lineno
                                context = self._extract_python_node_source(content, node)
                                
                                symbols[symbol_name] = {
                                    'type': 'variable',
                                    'line_no': line_no,
                                    'context': context
                                }

            self.import_relations[file_path] = imports
            self.module_symbols[file_path] = symbols

        except Exception as e:
            print(f"Error analyzing Python file {file_path}: {e}")

    def _extract_python_node_source(self, source: str, node) -> str:
        """Extract the source code for a Python AST node."""
        try:
            lines = source.splitlines()
            if hasattr(node, 'lineno') and hasattr(node, 'end_lineno'):
                start = node.lineno - 1
                end = getattr(node, 'end_lineno', start + 1)
                return '\n'.join(lines[start:end])
            return ""
        except Exception:
            return ""

    def _analyze_cpp_file(self, file_path: str, content: str) -> None:
        """Analyze a C/C++ file for includes and symbols."""
        imports = []
        symbols = {}
        
        # Process content line by line
        lines = content.splitlines()
        
        # Regular expressions for C/C++ code analysis
        include_pattern = re.compile(r'#include\s+[<"]([^>"]+)[>"]')
        class_pattern = re.compile(r'(?:class|struct)\s+(\w+)')
        function_pattern = re.compile(r'(\w+)\s*\([^)]*\)\s*(?:const|override|final|noexcept)?\s*(?:{|;)')
        namespace_pattern = re.compile(r'namespace\s+(\w+)')
        
        for line_no, line in enumerate(lines, 1):
            # Find include statements
            include_match = include_pattern.search(line)
            if include_match:
                imports.append((include_match.group(1), line_no))
            
            # Find class/struct definitions
            class_match = class_pattern.search(line)
            if class_match:
                class_name = class_match.group(1)
                context = self._get_context_around_line(file_path, line_no)
                symbols[class_name] = {
                    'type': 'class',
                    'line_no': line_no,
                    'context': context
                }
            
            # Find function definitions (simplified)
            function_match = function_pattern.search(line)
            if function_match and not line.strip().startswith('#') and not line.strip().startswith('//'):
                function_name = function_match.group(1)
                # Skip some common keywords that might be mistaken for functions
                if function_name not in ['if', 'while', 'for', 'switch', 'return']:
                    context = self._get_context_around_line(file_path, line_no)
                    symbols[function_name] = {
                        'type': 'function',
                        'line_no': line_no,
                        'context': context
                    }
            
            # Find namespace definitions
            namespace_match = namespace_pattern.search(line)
            if namespace_match:
                namespace_name = namespace_match.group(1)
                context = self._get_context_around_line(file_path, line_no)
                symbols[namespace_name] = {
                    'type': 'namespace',
                    'line_no': line_no,
                    'context': context
                }
        
        self.import_relations[file_path] = imports
        self.module_symbols[file_path] = symbols

    def _analyze_java_file(self, file_path: str, content: str) -> None:
        """Analyze a Java file for imports and symbols."""
        imports = []
        symbols = {}
        
        # Process content line by line
        lines = content.splitlines()
        
        # Regular expressions for Java code analysis
        package_pattern = re.compile(r'package\s+([\w.]+)')
        import_pattern = re.compile(r'import\s+([\w.]+(?:\.\*)?)')
        class_pattern = re.compile(r'(?:public|private|protected)?\s*(?:abstract|final)?\s*class\s+(\w+)')
        interface_pattern = re.compile(r'(?:public|private|protected)?\s*interface\s+(\w+)')
        method_pattern = re.compile(r'(?:public|private|protected)?\s*(?:static|final|abstract)?\s*(?:[\w<>[\],\s]+)\s+(\w+)\s*\([^)]*\)')
        
        for line_no, line in enumerate(lines, 1):
            # Find package declaration
            package_match = package_pattern.search(line)
            if package_match:
                package_name = package_match.group(1)
                imports.append((package_name, line_no))
            
            # Find import statements
            import_match = import_pattern.search(line)
            if import_match:
                import_name = import_match.group(1)
                imports.append((import_name, line_no))
            
            # Find class definitions
            class_match = class_pattern.search(line)
            if class_match:
                class_name = class_match.group(1)
                context = self._get_context_around_line(file_path, line_no)
                symbols[class_name] = {
                    'type': 'class',
                    'line_no': line_no,
                    'context': context
                }
            
            # Find interface definitions
            interface_match = interface_pattern.search(line)
            if interface_match:
                interface_name = interface_match.group(1)
                context = self._get_context_around_line(file_path, line_no)
                symbols[interface_name] = {
                    'type': 'interface',
                    'line_no': line_no,
                    'context': context
                }
            
            # Find method definitions
            method_match = method_pattern.search(line)
            if method_match:
                method_name = method_match.group(1)
                # Skip some common keywords that might be mistaken for methods
                if method_name not in ['if', 'while', 'for', 'switch', 'return']:
                    context = self._get_context_around_line(file_path, line_no)
                    symbols[method_name] = {
                        'type': 'method',
                        'line_no': line_no,
                        'context': context
                    }
        
        self.import_relations[file_path] = imports
        self.module_symbols[file_path] = symbols

    def _analyze_go_file(self, file_path: str, content: str) -> None:
        """Analyze a Go file for imports and symbols."""
        imports = []
        symbols = {}
        
        # Process content line by line
        lines = content.splitlines()
        
        # Regular expressions for Go code analysis
        package_pattern = re.compile(r'package\s+(\w+)')
        import_single_pattern = re.compile(r'import\s+"([^"]+)"')
        import_multi_start_pattern = re.compile(r'import\s+\(')
        import_multi_line_pattern = re.compile(r'\s*"([^"]+)"')
        func_pattern = re.compile(r'func\s+(?:\([^)]+\)\s+)?(\w+)')
        struct_pattern = re.compile(r'type\s+(\w+)\s+struct')
        interface_pattern = re.compile(r'type\s+(\w+)\s+interface')
        
        in_import_block = False
        
        for line_no, line in enumerate(lines, 1):
            # Find package declaration
            package_match = package_pattern.search(line)
            if package_match:
                package_name = package_match.group(1)
                imports.append((f"package {package_name}", line_no))
            
            # Handle single-line imports
            import_match = import_single_pattern.search(line)
            if import_match:
                import_name = import_match.group(1)
                imports.append((import_name, line_no))
            
            # Handle multi-line imports
            if import_multi_start_pattern.search(line):
                in_import_block = True
                continue
            
            if in_import_block:
                if line.strip() == ')':
                    in_import_block = False
                    continue
                    
                import_line_match = import_multi_line_pattern.search(line)
                if import_line_match:
                    import_name = import_line_match.group(1)
                    imports.append((import_name, line_no))
            
            # Find function definitions
            func_match = func_pattern.search(line)
            if func_match:
                func_name = func_match.group(1)
                context = self._get_context_around_line(file_path, line_no)
                symbols[func_name] = {
                    'type': 'function',
                    'line_no': line_no,
                    'context': context
                }
            
            # Find struct definitions
            struct_match = struct_pattern.search(line)
            if struct_match:
                struct_name = struct_match.group(1)
                context = self._get_context_around_line(file_path, line_no)
                symbols[struct_name] = {
                    'type': 'struct',
                    'line_no': line_no,
                    'context': context
                }
            
            # Find interface definitions
            interface_match = interface_pattern.search(line)
            if interface_match:
                interface_name = interface_match.group(1)
                context = self._get_context_around_line(file_path, line_no)
                symbols[interface_name] = {
                    'type': 'interface',
                    'line_no': line_no,
                    'context': context
                }
        
        self.import_relations[file_path] = imports
        self.module_symbols[file_path] = symbols

    def _find_references_in_file(self, file_path: str, content: str, language: str) -> None:
        """Find references to symbols in a file based on its language."""
        if language == "python":
            self._find_references_in_python_file(file_path, content)
        elif language == "cpp":
            self._find_references_in_cpp_file(file_path, content)
        elif language == "java":
            self._find_references_in_java_file(file_path, content)
        elif language == "go":
            self._find_references_in_go_file(file_path, content)

    def _find_references_in_python_file(self, file_path: str, content: str) -> None:
        """Find references to symbols in a Python file."""
        try:
            tree = ast.parse(content)
            
            for node in ast.walk(tree):
                # Find variable references
                if isinstance(node, ast.Name) and isinstance(node.ctx, ast.Load):
                    symbol_name = node.id
                    line_no = node.lineno
                    
                    # Track reference with context
                    if symbol_name not in self.symbol_references:
                        self.symbol_references[symbol_name] = []
                    
                    context = self._get_context_around_line(file_path, line_no)
                    self.symbol_references[symbol_name].append((file_path, line_no, context))
                
                # Find attribute references (e.g., obj.method())
                elif isinstance(node, ast.Attribute) and isinstance(node.ctx, ast.Load):
                    attr_name = node.attr
                    line_no = node.lineno
                    
                    if attr_name not in self.symbol_references:
                        self.symbol_references[attr_name] = []
                    
                    context = self._get_context_around_line(file_path, line_no)
                    self.symbol_references[attr_name].append((file_path, line_no, context))
        
        except Exception as e:
            print(f"Error finding references in Python file {file_path}: {e}")

    def _find_references_in_cpp_file(self, file_path: str, content: str) -> None:
        """Find references to symbols in a C/C++ file."""
        # Get all symbol names from all files to check for references
        all_symbols = set()
        for symbols_dict in self.module_symbols.values():
            all_symbols.update(symbols_dict.keys())
        
        # Process content line by line
        lines = content.splitlines()
        
        for line_no, line in enumerate(lines, 1):
            # Look for references to any known symbol
            for symbol_name in all_symbols:
                # Simple pattern matching (would be more robust with proper C++ parsing)
                pattern = r'\b' + re.escape(symbol_name) + r'\b'
                if re.search(pattern, line):
                    # Check if this is not the definition line
                    if (file_path in self.module_symbols and 
                        symbol_name in self.module_symbols[file_path] and 
                        self.module_symbols[file_path][symbol_name]['line_no'] == line_no):
                        continue
                    
                    if symbol_name not in self.symbol_references:
                        self.symbol_references[symbol_name] = []
                    
                    context = self._get_context_around_line(file_path, line_no)
                    self.symbol_references[symbol_name].append((file_path, line_no, context))

    def _find_references_in_java_file(self, file_path: str, content: str) -> None:
        """Find references to symbols in a Java file."""
        # Get all symbol names from all files to check for references
        all_symbols = set()
        for symbols_dict in self.module_symbols.values():
            all_symbols.update(symbols_dict.keys())
        
        # Process content line by line
        lines = content.splitlines()
        
        for line_no, line in enumerate(lines, 1):
            # Skip comment lines and import/package declarations
            if (line.strip().startswith("//") or 
                line.strip().startswith("/*") or 
                line.strip().startswith("import ") or 
                line.strip().startswith("package ")):
                continue
            
            # Look for references to any known symbol
            for symbol_name in all_symbols:
                # Simple pattern matching with word boundaries
                pattern = r'\b' + re.escape(symbol_name) + r'\b'
                if re.search(pattern, line):
                    # Check if this is not the definition line
                    if (file_path in self.module_symbols and 
                        symbol_name in self.module_symbols[file_path] and 
                        self.module_symbols[file_path][symbol_name]['line_no'] == line_no):
                        continue
                    
                    if symbol_name not in self.symbol_references:
                        self.symbol_references[symbol_name] = []
                    
                    context = self._get_context_around_line(file_path, line_no)
                    self.symbol_references[symbol_name].append((file_path, line_no, context))

    def _find_references_in_go_file(self, file_path: str, content: str) -> None:
        """Find references to symbols in a Go file."""
        # Get all symbol names from all files to check for references
        all_symbols = set()
        for symbols_dict in self.module_symbols.values():
            all_symbols.update(symbols_dict.keys())
        
        # Process content line by line
        lines = content.splitlines()
        
        for line_no, line in enumerate(lines, 1):
            # Skip comment lines and import/package declarations
            if (line.strip().startswith("//") or 
                line.strip().startswith("/*") or 
                line.strip().startswith("import ") or 
                line.strip().startswith("package ")):
                continue
            
            # Look for references to any known symbol
            for symbol_name in all_symbols:
                # Simple pattern matching with word boundaries
                pattern = r'\b' + re.escape(symbol_name) + r'\b'
                if re.search(pattern, line):
                    # Check if this is not the definition line
                    if (file_path in self.module_symbols and 
                        symbol_name in self.module_symbols[file_path] and 
                        self.module_symbols[file_path][symbol_name]['line_no'] == line_no):
                        continue
                    
                    if symbol_name not in self.symbol_references:
                        self.symbol_references[symbol_name] = []
                    
                    context = self._get_context_around_line(file_path, line_no)
                    self.symbol_references[symbol_name].append((file_path, line_no, context))

    def _build_symbol_index(self) -> None:
        """Build a comprehensive index of all symbols and where they're defined/used."""
        # Initialize the symbol index
        self.symbol_index = {}
        
        # First, add all symbol definitions
        for file_path, symbols in self.module_symbols.items():
            for symbol_name, details in symbols.items():
                if symbol_name not in self.symbol_index:
                    self.symbol_index[symbol_name] = []
                
                self.symbol_index[symbol_name].append({
                    'file': file_path,
                    'type': 'definition',
                    'symbol_type': details['type'],
                    'line_no': details['line_no'],
                    'context': details.get('context', ''),
                    'docstring': details.get('docstring', '')
                })
        
        # Then, add all references
        for symbol_name, references in self.symbol_references.items():
            if symbol_name not in self.symbol_index:
                self.symbol_index[symbol_name] = []
            
            for file_path, line_no, context in references:
                # Avoid duplicating references if they're already in definitions
                if not any(ref['file'] == file_path and ref['line_no'] == line_no and ref['type'] == 'definition' 
                          for ref in self.symbol_index.get(symbol_name, [])):
                    self.symbol_index[symbol_name].append({
                        'file': file_path,
                        'type': 'reference',
                        'line_no': line_no,
                        'context': context
                    })

    def build_graph(self) -> nx.DiGraph:
        """Build the NetworkX graph with enhanced node and edge information."""
        # We've already added basic file and directory nodes during parsing
        # Now add more detailed connections and data
        
        # Add nodes for all directories (if not already added)
        for directory in self.directories:
            if not self.graph.has_node(directory):
                self.graph.add_node(directory, type='directory')
            
            # Ensure parent directories exist and are connected
            parts = directory.split(os.sep)
            for i in range(1, len(parts)):
                parent_path = os.sep.join(parts[:i])
                if parent_path and not self.graph.has_node(parent_path):
                    self.graph.add_node(parent_path, type='directory')
                    self.directories.add(parent_path)
                
                # Connect parent to child directory
                if parent_path:
                    child_path = os.sep.join(parts[:i+1])
                    self.graph.add_edge(parent_path, child_path, edge_type='contains_directory')
        
        # Add nodes for all files with indices and code snippet nodes
        for file_path, file_idx in self.file_index.items():
            language = self._detect_language(file_path)
            
            # Update file node if it exists, create it otherwise
            if self.graph.has_node(file_path):
                self.graph.nodes[file_path].update({
                    'file_index': file_idx,
                    'directory': os.path.dirname(file_path),
                    'language': language
                })
            else:
                self.graph.add_node(file_path, 
                                   type='file',
                                   file_index=file_idx,
                                   directory=os.path.dirname(file_path),
                                   language=language)
            
            # Connect file to its directory
            directory = os.path.dirname(file_path)
            if directory:
                # Make sure the directory node exists
                if not self.graph.has_node(directory):
                    self.graph.add_node(directory, type='directory')
                    self.directories.add(directory)
                
                # Add edge from directory to file if it doesn't exist
                if not self.graph.has_edge(directory, file_path):
                    self.graph.add_edge(directory, file_path, edge_type='contains_file')
            
            # Create snippet nodes for the entire file
            if file_path in self.file_contents:
                chunks = self._chunk_code(self.file_contents[file_path])
                for idx, chunk_info in enumerate(chunks):
                    snippet_node = f"{file_path}::snippet::{idx}"
                    self.graph.add_node(snippet_node,
                                       type='snippet',
                                       code_snippet=chunk_info['code_snippet'],
                                       start_line=chunk_info['start_line'],
                                       end_line=chunk_info['end_line'],
                                       language=language)
                    # Connect file node to snippet node
                    self.graph.add_edge(file_path, snippet_node, 
                                       edge_type='contains_snippet',
                                       start_line=chunk_info['start_line'],
                                       end_line=chunk_info['end_line'])

            # Add nodes for symbols in this file
            for symbol, details in self.module_symbols.get(file_path, {}).items():
                symbol_node = f"{file_path}::{symbol}"
                self.graph.add_node(symbol_node, 
                                   type='symbol',
                                   symbol_type=details['type'],
                                   line_number=details['line_no'],
                                   context=details.get('context', ''),
                                   docstring=details.get('docstring', ''))
                self.graph.add_edge(file_path, symbol_node, 
                                   edge_type='defines',
                                   line_number=details['line_no'])

        # Add edges for imports with line numbers
        for file_path, imports in self.import_relations.items():
            for imp, line_no in imports:
                # Look for matching files or symbols
                for target_file, symbols in self.module_symbols.items():
                    if imp in symbols:
                        self.graph.add_edge(file_path, 
                                           f"{target_file}::{imp}",
                                           edge_type='import',
                                           line_number=line_no)
                    # For Python, handle module imports
                    elif self._detect_language(file_path) == "python" and target_file.replace('.py', '').endswith(imp):
                        self.graph.add_edge(file_path, 
                                           target_file,
                                           edge_type='import',
                                           line_number=line_no)
                    # For Java, handle package imports
                    elif self._detect_language(file_path) == "java" and imp.startswith(os.path.splitext(os.path.basename(target_file))[0]):
                        self.graph.add_edge(file_path, 
                                           target_file,
                                           edge_type='import',
                                           line_number=line_no)
        
        # Add edges for symbol references
        for symbol, references in self.symbol_references.items():
            for file_path, line_no, context in references:
                # Find symbol nodes that match this reference
                for target_file, symbols in self.module_symbols.items():
                    if symbol in symbols:
                        # Create reference edge
                        self.graph.add_edge(file_path, 
                                           f"{target_file}::{symbol}",
                                           edge_type='references',
                                           line_number=line_no,
                                           context=context)
        
        return self.graph

    def export_to_arango(self, url: str, username: str, password: str, db_name: str = "codebase", 
                         graph_name: str = "Custom_Flask", node_collection: str = "nodes", 
                         edge_collection: str = "edges", overwrite: bool = False) -> None:
        """
        Export the NetworkX graph to ArangoDB.
        
        Args:
            url: ArangoDB server URL
            username: ArangoDB username
            password: ArangoDB password
            db_name: Database name
            graph_name: Graph name
            node_collection: Node collection name
            edge_collection: Edge collection name
            overwrite: Whether to overwrite existing database
        """
        # Initialize ArangoDB client
        client = ArangoClient(hosts=url)
        sys_db = client.db('_system', username=username, password=password)
        
        # Create or use existing database
        if sys_db.has_database(db_name):
            if overwrite:
                sys_db.delete_database(db_name)
                sys_db.create_database(db_name)
                print(f"Database '{db_name}' recreated.")
            else:
                print(f"Using existing database '{db_name}'.")
        else:
            sys_db.create_database(db_name)
            print(f"Database '{db_name}' created.")
        
        # Connect to the database
        db = client.db(db_name, username=username, password=password)
        
        # Create or use existing collections
        if db.has_collection(node_collection):
            nodes = db.collection(node_collection)
            nodes.truncate()
        else:
            nodes = db.create_collection(node_collection)
        
        if db.has_collection(edge_collection):
            edges = db.collection(edge_collection)
            edges.truncate()
        else:
            edges = db.create_edge_collection(edge_collection)
        
        # Create or use existing graph
        if db.has_graph(graph_name):
            graph = db.graph(graph_name)
        else:
            graph = db.create_graph(graph_name)
            # Define edge definition
            graph.create_edge_definition(
                edge_collection=edge_collection,
                from_vertex_collections=[node_collection],
                to_vertex_collections=[node_collection]
            )
        
        # Prepare nodes for ArangoDB (ensuring unique IDs)
        node_mapping = {}  # Maps node names to ArangoDB keys
        
        # Add nodes to ArangoDB
        print("Adding nodes to ArangoDB...")
        for node_name, node_attrs in self.graph.nodes(data=True):
            # Create a sanitized key for ArangoDB
            key = re.sub(r'[^a-zA-Z0-9_\-]', '_', node_name)
            node_mapping[node_name] = key
            
            # Include all attributes and the original node name
            node_data = {
                '_key': key,
                'original_name': node_name
            }
            node_data.update(node_attrs)
            
            # Handle special data types for ArangoDB
            for attr, value in node_data.items():
                if isinstance(value, (set, tuple)):
                    node_data[attr] = list(value)
            
            # Insert the node
            nodes.insert(node_data)
        
        # Add edges to ArangoDB
        print("Adding edges to ArangoDB...")
        for src, dst, edge_attrs in self.graph.edges(data=True):
            # Create edge with proper from/to
            edge_data = {
                '_from': f"{node_collection}/{node_mapping[src]}",
                '_to': f"{node_collection}/{node_mapping[dst]}"
            }
            edge_data.update(edge_attrs)
            
            # Handle special data types for ArangoDB
            for attr, value in edge_data.items():
                if isinstance(value, (set, tuple)):
                    edge_data[attr] = list(value)
            
            # Insert the edge
            edges.insert(edge_data)
        
        print(f"Exported graph to ArangoDB: {len(self.graph.nodes())} nodes and {len(self.graph.edges())} edges.")

    def query_database(self, url: str, username: str, password: str, db_name: str = "codebase", 
                      query: str = None) -> List[Dict]:
        """
        Execute a query against the ArangoDB database.
        
        Args:
            url: ArangoDB server URL
            username: ArangoDB username
            password: ArangoDB password
            db_name: Database name
            query: AQL query string
            
        Returns:
            Query results as a list of dictionaries
        """
        client = ArangoClient(hosts=url)
        db = client.db(db_name, username=username, password=password)
        
        if query is None:
            # Default query to get basic statistics
            query = """
            RETURN {
                "node_count": LENGTH(FOR v IN nodes RETURN v),
                "edge_count": LENGTH(FOR e IN edges RETURN e),
                "file_count": LENGTH(FOR v IN nodes FILTER v.type == 'file' RETURN v),
                "directory_count": LENGTH(FOR v IN nodes FILTER v.type == 'directory' RETURN v),
                "symbol_count": LENGTH(FOR v IN nodes FILTER v.type == 'symbol' RETURN v)
            }
            """
        
        cursor = db.aql.execute(query)
        return [doc for doc in cursor]

    def export_to_json(self, output_path: str) -> None:
        """
        Export the graph data to a JSON file for backup or analysis outside ArangoDB.
        
        Args:
            output_path: Path to write the JSON file
        """
        data = {
            "nodes": [],
            "edges": []
        }
        
        # Export nodes
        for node_name, attrs in self.graph.nodes(data=True):
            node_data = {"id": node_name}
            node_data.update(attrs)
            data["nodes"].append(node_data)
        
        # Export edges
        for src, dst, attrs in self.graph.edges(data=True):
            edge_data = {
                "source": src,
                "target": dst
            }
            edge_data.update(attrs)
            data["edges"].append(edge_data)
        
        # Write to file
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        
        print(f"Exported graph to JSON file: {output_path}")

    def analyze_codebase(self) -> Dict[str, any]:
        """
        Perform basic analysis on the codebase and return statistics.
        
        Returns:
            Dictionary with analysis results
        """
        stats = {
            "total_files": len(self.file_index),
            "total_directories": len(self.directories),
            "total_symbols": sum(len(symbols) for symbols in self.module_symbols.values()),
            "languages": {},
            "file_sizes": {
                "min": float('inf'),
                "max": 0,
                "avg": 0
            },
            "symbol_types": {}
        }
        
        # Count files by language
        for file_path in self.file_index:
            lang = self._detect_language(file_path)
            stats["languages"][lang] = stats["languages"].get(lang, 0) + 1
            
            # Track file sizes
            file_size = len(self.file_contents.get(file_path, ""))
            stats["file_sizes"]["min"] = min(stats["file_sizes"]["min"], file_size)
            stats["file_sizes"]["max"] = max(stats["file_sizes"]["max"], file_size)
        
        # Calculate average file size
        if stats["total_files"] > 0:
            total_size = sum(len(content) for content in self.file_contents.values())
            stats["file_sizes"]["avg"] = total_size / stats["total_files"]
        else:
            stats["file_sizes"]["min"] = 0
        
        # Count symbols by type
        for symbols in self.module_symbols.values():
            for symbol, details in symbols.items():
                symbol_type = details.get("type", "unknown")
                stats["symbol_types"][symbol_type] = stats["symbol_types"].get(symbol_type, 0) + 1
        
        return stats

    def run_workflow(self, code_path: str, arango_url: str, username: str, password: str, 
                    db_name: str = "codebase") -> Dict:
        """
        Run the complete workflow: parse files, build graph, export to ArangoDB, and analyze.
        
        Args:
            code_path: Path to the codebase
            arango_url: ArangoDB server URL
            username: ArangoDB username
            password: ArangoDB password
            db_name: Database name
            
        Returns:
            Analysis results
        """
        print(f"Processing codebase at: {code_path}")
        
        # Parse files
        self.parse_files()
        print(f"Parsed {len(self.file_index)} files and {len(self.directories)} directories")
        
        # Build graph
        self.build_graph()
        print(f"Built graph with {len(self.graph.nodes())} nodes and {len(self.graph.edges())} edges")
        
        # Export to ArangoDB
        self.export_to_arango(
            url=arango_url,
            username=username,
            password=password,
            db_name=db_name,
            overwrite=True
        )
        
        # Analyze codebase
        analysis = self.analyze_codebase()
        print(f"Analysis complete: {analysis}")
        
        return analysis
    
    def validate_graph_and_data(self) -> dict:
        """
        Validate the parsed data and graph construction.
        Returns a detailed report on what was found and potential issues.
        """
        report = {
            "files": {
                "count": len(self.file_index),
                "samples": list(self.file_index.keys())[:5],  # First 5 files
                "extensions": {}
            },
            "directories": {
                "count": len(self.directories),
                "samples": list(self.directories)[:5]  # First 5 directories
            },
            "symbols": {
                "count": sum(len(symbols) for symbols in self.module_symbols.values()),
                "by_type": {},
                "samples": []
            },
            "graph": {
                "nodes": self.graph.number_of_nodes(),
                "edges": self.graph.number_of_edges(),
                "node_types": {},
                "edge_types": {}
            },
            "possible_issues": []
        }
        
        # Check file extensions
        for file_path in self.file_index:
            _, ext = os.path.splitext(file_path)
            ext = ext.lower()
            report["files"]["extensions"][ext] = report["files"]["extensions"].get(ext, 0) + 1
        
        # Check for supported extensions
        supported_exts = []
        for lang, exts in self.language_extensions.items():
            supported_exts.extend(exts)
        
        if set(report["files"]["extensions"].keys()).isdisjoint(supported_exts):
            report["possible_issues"].append("No files with supported extensions found.")
        
        # Check symbol types
        for file_path, symbols in self.module_symbols.items():
            for symbol_name, details in symbols.items():
                symbol_type = details['type']
                report["symbols"]["by_type"][symbol_type] = report["symbols"]["by_type"].get(symbol_type, 0) + 1
                
                if len(report["symbols"]["samples"]) < 5:
                    report["symbols"]["samples"].append({
                        "name": symbol_name,
                        "file": file_path,
                        "type": symbol_type,
                        "line": details['line_no']
                    })
        
        # Check graph node and edge types
        for _, data in self.graph.nodes(data=True):
            node_type = data.get('type', 'unknown')
            report["graph"]["node_types"][node_type] = report["graph"]["node_types"].get(node_type, 0) + 1
        
        for _, _, data in self.graph.edges(data=True):
            edge_type = data.get('edge_type', 'unknown')
            report["graph"]["edge_types"][edge_type] = report["graph"]["edge_types"].get(edge_type, 0) + 1
        
        # Check if nodes match files and directories
        if report["graph"]["node_types"].get("file", 0) != report["files"]["count"]:
            report["possible_issues"].append(
                f"Mismatch between file count ({report['files']['count']}) and file nodes in graph ({report['graph']['node_types'].get('file', 0)})"
            )
        
        if report["graph"]["node_types"].get("directory", 0) != report["directories"]["count"]:
            report["possible_issues"].append(
                f"Mismatch between directory count ({report['directories']['count']}) and directory nodes in graph ({report['graph']['node_types'].get('directory', 0)})"
            )
        
        # Check if symbols have corresponding nodes
        symbol_count = report["symbols"]["count"]
        symbol_nodes = report["graph"]["node_types"].get("symbol", 0)
        if symbol_count != symbol_nodes:
            report["possible_issues"].append(
                f"Mismatch between symbol count ({symbol_count}) and symbol nodes in graph ({symbol_nodes})"
            )
        
        # Validate directory structure
        if report["directories"]["count"] > 0 and report["files"]["count"] > 0:
            # Check if files are connected to their directories
            contains_file_edges = report["graph"]["edge_types"].get("contains_file", 0)
            if contains_file_edges < report["files"]["count"]:
                report["possible_issues"].append(
                    f"Some files may not be properly connected to their directories ({contains_file_edges} edges for {report['files']['count']} files)"
                )
        
        return report

In [26]:
# Example usage
visualizer = CodebaseVisualizer(root_dir="flask")
visualizer.parse_files()
G = visualizer.build_graph()
print(f"Graph has {len(G.nodes())} nodes and {len(G.edges())} edges")

report = visualizer.validate_graph_and_data()
print(json.dumps(report, indent=2))

Graph has 2974 nodes and 8795 edges
{
  "files": {
    "count": 83,
    "samples": [
      "tests/test_basic.py",
      "tests/conftest.py",
      "tests/test_converters.py",
      "tests/test_logging.py",
      "tests/test_signals.py"
    ],
    "extensions": {
      ".py": 83
    }
  },
  "directories": {
    "count": 68,
    "samples": [
      "docs/patterns",
      "examples/javascript/tests",
      "examples/tutorial/flaskr/templates/blog",
      "examples/javascript/js_example",
      "tests/test_apps/blueprintapp"
    ]
  },
  "symbols": {
    "count": 1893,
    "by_type": {
      "variable": 660,
      "function": 1097,
      "class": 136
    },
    "samples": [
      {
        "name": "require_cpython_gc",
        "file": "tests/test_basic.py",
        "type": "variable",
        "line": 24
      },
      {
        "name": "test_options_work",
        "file": "tests/test_basic.py",
        "type": "function",
        "line": 30
      },
      {
        "name": "test_options_on

In [22]:
#visualizer.export_to_arango(db_name="FlaskRepv1",username="root",password="cUZ0YaNdcwfUTw6VjRny",host="https://d2eeb8083350.arangodb.cloud:8529")


In [23]:
# visualizer = CodebaseVisualizer("flask")
# visualizer.parse_files()
# G = visualizer.build_graph()

In [24]:
# class GraphVisualizer:
#     def __init__(self, graph: nx.Graph):
#         self.graph = graph
#         self.pos = None
        
#     def set_layout(self, layout_type: str = 'spring', **layout_params) -> None:
#         """
#         Set the layout for the graph visualization.
        
#         Args:
#             layout_type: Type of layout ('spring', 'circular', 'kamada_kawai', 
#                         'random', 'shell', 'spectral')
#             layout_params: Additional parameters for the layout algorithm
#         """
#         layout_funcs = {
#             'spring': nx.spring_layout,
#             'circular': nx.circular_layout,
#             'kamada_kawai': nx.kamada_kawai_layout,
#             'random': nx.random_layout,
#             'shell': nx.shell_layout,
#             'spectral': nx.spectral_layout
#         }
        
#         if layout_type not in layout_funcs:
#             raise ValueError(f"Unsupported layout type. Choose from: {list(layout_funcs.keys())}")
            
#         self.pos = layout_funcs[layout_type](self.graph, **layout_params)
    
#     def _get_node_colors(self) -> Dict[str, str]:
#         """Extract node colors from graph attributes or generate defaults."""
#         colors = {}
#         for node in self.graph.nodes():
#             # Check for color in node attributes
#             attrs = self.graph.nodes[node]
#             if 'fillcolor' in attrs:
#                 colors[node] = attrs['fillcolor']
#             elif 'color' in attrs:
#                 colors[node] = attrs['color']
#             else:
#                 colors[node] = 'lightblue'  # default color
#         return colors
    
#     def _get_node_sizes(self) -> Dict[str, float]:
#         """Extract or compute node sizes."""
#         sizes = {}
#         for node in self.graph.nodes():
#             attrs = self.graph.nodes[node]
#             if 'size' in attrs:
#                 sizes[node] = attrs['size']
#             else:
#                 # Default size based on node degree
#                 sizes[node] = 1000 * (1 + self.graph.degree(node) / 10)
#         return sizes
    
#     def _get_edge_colors(self) -> Dict[Tuple[str, str], str]:
#         """Extract edge colors from graph attributes or generate defaults."""
#         colors = {}
#         for u, v in self.graph.edges():
#             edge_data = self.graph.get_edge_data(u, v)
#             if 'color' in edge_data:
#                 colors[(u, v)] = edge_data['color']
#             else:
#                 colors[(u, v)] = 'gray'  # default color
#         return colors
    
#     def _get_node_labels(self) -> Dict[str, str]:
#         """Extract node labels from graph attributes."""
#         labels = {}
#         for node in self.graph.nodes():
#             attrs = self.graph.nodes[node]
#             if 'label' in attrs:
#                 labels[node] = attrs['label']
#             else:
#                 labels[node] = str(node)
#         return labels
    
#     def _get_edge_labels(self) -> Dict[Tuple[str, str], str]:
#         """Extract edge labels from graph attributes."""
#         labels = {}
#         for u, v in self.graph.edges():
#             edge_data = self.graph.get_edge_data(u, v)
#             if 'label' in edge_data:
#                 labels[(u, v)] = edge_data['label']
#         return labels

#     def visualize(self, 
#                  figsize: Tuple[int, int] = (12, 8),
#                  node_size: Optional[Dict[str, float]] = None,
#                  node_color: Optional[Dict[str, str]] = None,
#                  edge_color: Optional[Dict[Tuple[str, str], str]] = None,
#                  with_labels: bool = True,
#                  font_size: int = 8,
#                  title: Optional[str] = None,
#                  show_edge_labels: bool = True,
#                  alpha: float = 0.7,
#                  save_path: Optional[str] = None) -> None:
#         """
#         Visualize the graph with customizable options.
        
#         Args:
#             figsize: Size of the figure (width, height)
#             node_size: Dictionary mapping nodes to their sizes
#             node_color: Dictionary mapping nodes to their colors
#             edge_color: Dictionary mapping edges to their colors
#             with_labels: Whether to show node labels
#             font_size: Size of the font for labels
#             title: Title of the graph
#             show_edge_labels: Whether to show edge labels
#             alpha: Transparency of nodes
#             save_path: Path to save the visualization (if None, displays instead)
#         """
#         if self.pos is None:
#             self.set_layout('spring')
            
#         plt.figure(figsize=figsize)
        
#         # Get or use provided node attributes
#         node_colors = node_color if node_color is not None else self._get_node_colors()
#         node_sizes = node_size if node_size is not None else self._get_node_sizes()
#         edge_colors = edge_color if edge_color is not None else self._get_edge_colors()
        
#         # Draw nodes
#         nx.draw_networkx_nodes(self.graph, self.pos,
#                              node_color=[node_colors[node] for node in self.graph.nodes()],
#                              node_size=[node_sizes[node] for node in self.graph.nodes()],
#                              alpha=alpha)
        
#         # Draw edges
#         for (u, v) in self.graph.edges():
#             nx.draw_networkx_edges(self.graph, self.pos,
#                                  edgelist=[(u, v)],
#                                  edge_color=edge_colors.get((u, v), 'gray'),
#                                  alpha=0.5)
        
#         # Add labels if requested
#         if with_labels:
#             labels = self._get_node_labels()
#             nx.draw_networkx_labels(self.graph, self.pos, labels,
#                                   font_size=font_size)
        
#         # Add edge labels if requested
#         if show_edge_labels:
#             edge_labels = self._get_edge_labels()
#             if edge_labels:
#                 nx.draw_networkx_edge_labels(self.graph, self.pos,
#                                            edge_labels=edge_labels,
#                                            font_size=font_size-2)
        
#         if title:
#             plt.title(title)
        
#         plt.axis('off')
        
#         if save_path:
#             plt.savefig(save_path, bbox_inches='tight', dpi=300)
#             plt.close()
#         else:
#             plt.show()

# # Example usage:
# '''
# # Create a sample graph
# G = nx.Graph()
# G.add_nodes_from([
#     (1, {'fillcolor': 'lightblue', 'label': 'Node 1'}),
#     (2, {'fillcolor': 'lightgreen', 'label': 'Node 2'}),
#     (3, {'fillcolor': 'lightred', 'label': 'Node 3'})
# ])
# G.add_edges_from([
#     (1, 2, {'color': 'blue', 'label': 'Edge 1-2'}),
#     (2, 3, {'color': 'red', 'label': 'Edge 2-3'})
# ])
# '''
# # Create visualizer and display graph
# visualizer = GraphVisualizer(G)
# visualizer.set_layout('spring', k=2)  # k controls the spacing between nodes
# visualizer.visualize(
#     figsize=(10, 8),
#     font_size=10,
#     title="Sample Graph Visualization",
# )


In [25]:
from pyvis.network import Network

def visualize_codebase_graph(codebase_path, output_html="codebase_graph.html", limit_nodes=None):
    """
    Visualize the codebase graph using pyvis.
    
    Args:
        codebase_path: Path to the codebase directory
        output_html: Output HTML file for the visualization/
        limit_nodes: Optional limit on the number of nodes to display (for large codebases)
    """
    # Initialize and build the graph
    visualizer = CodebaseVisualizer(codebase_path)
    visualizer.parse_files()
    graph = visualizer.build_graph()
    
    # Export graph to JSON (optional)
    visualizer.export_graph_json('codebase_graph.json')
    
    # Create a pyvis network
    net = Network(height="900px", width="100%", bgcolor="#222222", font_color="white")
    
    # Configure physics
    net.barnes_hut(gravity=-5000, central_gravity=0.3, spring_length=200)
    
    # Define node groups and colors
    node_colors = {
        'file': '#4287f5',
        'directory': '#42f5a7',
        'symbol': '#f542cb',
        'snippet': '#f5a742'
    }
    
    # If we need to limit nodes for performance
    if limit_nodes and len(graph.nodes()) > limit_nodes:
        # Focus on file and directory nodes, and limit symbol nodes
        important_nodes = [node for node, data in graph.nodes(data=True) 
                          if data.get('type') in ['file', 'directory']]
        
        # Add some symbol nodes to reach the limit
        symbols = [node for node, data in graph.nodes(data=True) 
                  if data.get('type') == 'symbol']
        
        # Take a subset of symbols based on connectivity
        symbol_importance = sorted(
            [(node, graph.degree(node)) for node in symbols],
            key=lambda x: x[1],
            reverse=True
        )
        
        important_symbols = [node for node, _ in symbol_importance[:limit_nodes - len(important_nodes)]]
        selected_nodes = important_nodes + important_symbols
        
        # Create a subgraph
        graph = graph.subgraph(selected_nodes)
    
    # Add nodes with appropriate styles
    for node, node_data in graph.nodes(data=True):
        node_type = node_data.get('type', 'unknown')
        label = os.path.basename(node) if '/' in node else node
        
        # Truncate very long labels
        if len(label) > 30:
            label = label[:27] + "..."
        
        # Create hover title with more details
        title = f"<div style='max-width:300px;'>"
        title += f"<b>{node}</b><br>"
        title += f"Type: {node_type}<br>"
        
        if node_type == 'file':
            title += f"Directory: {node_data.get('directory', 'N/A')}<br>"
            
        elif node_type == 'symbol':
            title += f"Symbol type: {node_data.get('symbol_type', 'N/A')}<br>"
            title += f"Line: {node_data.get('line_number', 'N/A')}<br>"
            
            if node_data.get('docstring'):
                docstring = node_data['docstring']
                if len(docstring) > 200:
                    docstring = docstring[:197] + "..."
                title += f"Docstring: {docstring}<br>"
                
        elif node_type == 'snippet':
            title += f"Lines: {node_data.get('start_line', 'N/A')}-{node_data.get('end_line', 'N/A')}<br>"
            
            if node_data.get('code_snippet'):
                snippet = node_data['code_snippet'].replace('\n', '<br>')
                if len(snippet) > 300:
                    snippet = snippet[:297] + "..."
                title += f"Code:<br><pre>{snippet}</pre>"
                
        title += "</div>"
        
        # Determine node size based on type and connections
        size = 15  # Default size
        if node_type == 'directory':
            size = 25
        elif node_type == 'file':
            size = 20
        elif node_type == 'symbol' and node_data.get('symbol_type') == 'class':
            size = 18
        
        # Add node with appropriate styling
        net.add_node(
            node, 
            label=label, 
            title=title,
            color=node_colors.get(node_type, '#999999'),
            size=size,
            shape='dot' if node_type != 'directory' else 'diamond'
        )
    
    # Add edges with appropriate styles
    for source, target, edge_data in graph.edges(data=True):
        edge_type = edge_data.get('edge_type', 'unknown')
        
        # Style edges differently based on type
        if edge_type == 'import':
            color = '#f5f542'
            width = 2
            dashes = False
        elif edge_type == 'references':
            color = '#f54242'
            width = 1
            dashes = [5, 5]
        elif edge_type == 'defines':
            color = '#42f55a'
            width = 3
            dashes = False
        elif edge_type == 'contains_snippet':
            color = '#42c8f5'
            width = 1
            dashes = [2, 2]
        else:
            color = '#999999'
            width = 1
            dashes = False
        
        # Create hover title with edge details
        title = f"<div><b>{edge_type}</b><br>"
        
        if edge_data.get('line_number'):
            title += f"Line: {edge_data['line_number']}<br>"
            
        if edge_data.get('context'):
            context = edge_data['context']
            if len(context) > 200:
                context = context[:197] + "..."
            title += f"Context: {context}"
            
        title += "</div>"
        
        # Add edge with styling
        net.add_edge(
            source, 
            target, 
            title=title,
            color=color,
            width=width,
            dashes=dashes
        )
    
    # Enable physics, navigation and interaction options
    net.toggle_physics(True)
    net.show_buttons(filter_=['physics'])
    
    # Save the visualization
    net.save_graph(output_html)
    print(f"Graph visualization saved to {output_html}")
    
    return output_html

def visualize_symbol_subgraph(visualizer, symbol_name, output_html="symbol_graph.html"):
    """
    Create a focused visualization of a symbol and its relationships.
    
    Args:
        visualizer: Initialized CodebaseVisualizer instance
        symbol_name: Name of the symbol to visualize
        output_html: Output HTML file for the visualization
    """
    # Get the full graph
    full_graph = visualizer.graph
    
    # Find all nodes related to this symbol
    symbol_nodes = [node for node in full_graph.nodes() if f"::{symbol_name}" in node]
    
    if not symbol_nodes:
        print(f"Symbol '{symbol_name}' not found in the graph.")
        return None
    
    # Get nodes that are connected to symbol nodes (1-hop neighborhood)
    related_nodes = set(symbol_nodes)
    for node in symbol_nodes:
        # Add predecessors (nodes that reference this symbol)
        related_nodes.update(full_graph.predecessors(node))
        # Add successors (nodes that this symbol references)
        related_nodes.update(full_graph.successors(node))
    
    # Create a subgraph
    subgraph = full_graph.subgraph(related_nodes)
    
    # Create a pyvis network
    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    net.barnes_hut(gravity=-2000, central_gravity=0.3, spring_length=150)
    
    # Node colors by type
    node_colors = {
        'file': '#4287f5',
        'directory': '#42f5a7',
        'symbol': '#f542cb',
        'snippet': '#f5a742'
    }
    
    # Add nodes with appropriate styles
    for node, node_data in subgraph.nodes(data=True):
        node_type = node_data.get('type', 'unknown')
        label = os.path.basename(node) if '/' in node else node
        
        # Make the focus symbol nodes larger and highlighted
        if node in symbol_nodes:
            size = 30
            color = '#ff0000'  # Bright red for focus
        else:
            size = 15
            color = node_colors.get(node_type, '#999999')
        
        # Add hover information
        title = f"<div style='max-width:300px;'>"
        title += f"<b>{node}</b><br>"
        title += f"Type: {node_type}<br>"
        
        if node_type == 'symbol':
            title += f"Symbol type: {node_data.get('symbol_type', 'N/A')}<br>"
            if node_data.get('docstring'):
                title += f"Docstring: {node_data.get('docstring', '')}<br>"
                
        title += "</div>"
        
        # Add node with styling
        net.add_node(
            node,
            label=label,
            title=title,
            color=color,
            size=size
        )
    
    # Add edges with styles
    for source, target, edge_data in subgraph.edges(data=True):
        edge_type = edge_data.get('edge_type', 'unknown')
        
        # Style edges by type
        if edge_type == 'import':
            color = '#f5f542'  # Yellow
        elif edge_type == 'references':
            color = '#f54242'  # Red
        elif edge_type == 'defines':
            color = '#42f55a'  # Green
        else:
            color = '#999999'  # Gray
        
        # Add edge with details in hover
        title = f"{edge_type}"
        if edge_data.get('line_number'):
            title += f" (line {edge_data['line_number']})"
            
        net.add_edge(source, target, title=title, color=color)
    
    # Enable physics and navigation
    net.toggle_physics(True)
    net.show_buttons(filter_=['physics'])
    
    # Save the visualization
    net.save_graph(output_html)
    print(f"Symbol graph visualization saved to {output_html}")
    
    return output_html


### Visualize graph

In [1]:
# # Visualize the entire codebase graph (with node limit for performance)
# visualize_codebase_graph("flask", limit_nodes=200)

# # Visualize a specific symbol (like "has_level_handler")
# visualizer = CodebaseVisualizer("flask")
# visualizer.parse_files()
# G = visualizer.build_graph()

NameError: name 'visualize_codebase_graph' is not defined

### Make Database


In [14]:

db = ArangoClient(hosts="https://d2eeb8083350.arangodb.cloud:8529").db(username="root", password="cUZ0YaNdcwfUTw6VjRny", verify=True)

print(db)

<StandardDatabase _system>


In [16]:

G_adb = nxadb.Graph(
    name="FlaskRepv1",
    db=db,
    incoming_graph_data=G,
    write_batch_size=50000, # feel free to modify
    overwrite_graph=True
)

print(G_adb)

[19:38:16 +0530] [INFO]: Overwriting graph 'FlaskRepv1'
[19:38:17 +0530] [INFO]: Graph 'FlaskRepv1' exists.
[19:38:17 +0530] [INFO]: Default node type set to 'FlaskRepv1_node'
[2025/03/09 19:38:19 +0530] [41950] [INFO] - adbnx_adapter: Instantiated ADBNX_Adapter with database '_system'


ReadTimeout: HTTPSConnectionPool(host='d2eeb8083350.arangodb.cloud', port=8529): Read timed out. (read timeout=60)

### Run from loaded database

In [28]:
db = ArangoClient(hosts="https://d2eeb8083350.arangodb.cloud:8529").db(username="root", password="cUZ0YaNdcwfUTw6VjRny", verify=True)

print(db)

<StandardDatabase _system>


In [29]:
G_adb = nxadb.Graph(
    name="FlaskRepv1",
    db=db,
    #incoming_graph_data=G,
    #write_batch_size=50000 # feel free to modify
)

print(type(G_adb))

[20:27:38 +0530] [INFO]: Graph 'FlaskRepv1' exists.
[20:27:39 +0530] [INFO]: Default node type set to 'FlaskRepv1_node'


<class 'nx_arangodb.classes.graph.Graph'>


In [30]:
arango_graph = ArangoGraph(db)

In [32]:
print( arango_graph.schema )

{'Graph Schema': [{'graph_name': 'CodebaseGraph', 'edge_definitions': [{'edge_collection': 'CodebaseGraph_node_to_CodebaseGraph_node', 'from_vertex_collections': ['CodebaseGraph_node'], 'to_vertex_collections': ['CodebaseGraph_node']}]}, {'graph_name': 'FlaskRepv1_node_to_FlaskRespv1_node', 'edge_definitions': [{'edge_collection': 'FlaskRepv1_node_to_FlaskRespv1_node_node_to_FlaskRepv1_node_to_FlaskRespv1_node_node', 'from_vertex_collections': ['FlaskRepv1_node_to_FlaskRespv1_node_node'], 'to_vertex_collections': ['FlaskRepv1_node_to_FlaskRespv1_node_node']}]}, {'graph_name': 'FlaskRepv1_node', 'edge_definitions': [{'edge_collection': 'FlaskRepv1_node_node_to_FlaskRepv1_node_node', 'from_vertex_collections': ['FlaskRepv1_node_node'], 'to_vertex_collections': ['FlaskRepv1_node_node']}]}, {'graph_name': 'code_graph', 'edge_definitions': [{'edge_collection': 'code_edges', 'from_vertex_collections': ['code_nodes'], 'to_vertex_collections': ['code_nodes']}]}, {'graph_name': 'FlaskRespv1', '

In [ ]:
from pyvis.network import Network

def visualize_codebase_graph(codebase_path, output_html="codebase_graph.html", limit_nodes=None):
    """
    Visualize the codebase graph using pyvis.
    
    Args:
        codebase_path: Path to the codebase directory
        output_html: Output HTML file for the visualization/
        limit_nodes: Optional limit on the number of nodes to display (for large codebases)
    """
    # Initialize and build the graph
    visualizer = CodebaseVisualizer(codebase_path)
    visualizer.parse_files()
    graph = visualizer.build_graph()
    
    # Export graph to JSON (optional)
    visualizer.export_graph_json('codebase_graph.json')
    
    # Create a pyvis network
    net = Network(height="900px", width="100%", bgcolor="#222222", font_color="white")
    
    # Configure physics
    net.barnes_hut(gravity=-5000, central_gravity=0.3, spring_length=200)
    
    # Define node groups and colors
    node_colors = {
        'file': '#4287f5',
        'directory': '#42f5a7',
        'symbol': '#f542cb',
        'snippet': '#f5a742'
    }
    
    # If we need to limit nodes for performance
    if limit_nodes and len(graph.nodes()) > limit_nodes:
        # Focus on file and directory nodes, and limit symbol nodes
        important_nodes = [node for node, data in graph.nodes(data=True) 
                          if data.get('type') in ['file', 'directory']]
        
        # Add some symbol nodes to reach the limit
        symbols = [node for node, data in graph.nodes(data=True) 
                  if data.get('type') == 'symbol']
        
        # Take a subset of symbols based on connectivity
        symbol_importance = sorted(
            [(node, graph.degree(node)) for node in symbols],
            key=lambda x: x[1],
            reverse=True
        )
        
        important_symbols = [node for node, _ in symbol_importance[:limit_nodes - len(important_nodes)]]
        selected_nodes = important_nodes + important_symbols
        
        # Create a subgraph
        graph = graph.subgraph(selected_nodes)
    
    # Add nodes with appropriate styles
    for node, node_data in graph.nodes(data=True):
        node_type = node_data.get('type', 'unknown')
        label = os.path.basename(node) if '/' in node else node
        
        # Truncate very long labels
        if len(label) > 30:
            label = label[:27] + "..."
        
        # Create hover title with more details
        title = f"<div style='max-width:300px;'>"
        title += f"<b>{node}</b><br>"
        title += f"Type: {node_type}<br>"
        
        if node_type == 'file':
            title += f"Directory: {node_data.get('directory', 'N/A')}<br>"
            
        elif node_type == 'symbol':
            title += f"Symbol type: {node_data.get('symbol_type', 'N/A')}<br>"
            title += f"Line: {node_data.get('line_number', 'N/A')}<br>"
            
            if node_data.get('docstring'):
                docstring = node_data['docstring']
                if len(docstring) > 200:
                    docstring = docstring[:197] + "..."
                title += f"Docstring: {docstring}<br>"
                
        elif node_type == 'snippet':
            title += f"Lines: {node_data.get('start_line', 'N/A')}-{node_data.get('end_line', 'N/A')}<br>"
            
            if node_data.get('code_snippet'):
                snippet = node_data['code_snippet'].replace('\n', '<br>')
                if len(snippet) > 300:
                    snippet = snippet[:297] + "..."
                title += f"Code:<br><pre>{snippet}</pre>"
                
        title += "</div>"
        
        # Determine node size based on type and connections
        size = 15  # Default size
        if node_type == 'directory':
            size = 25
        elif node_type == 'file':
            size = 20
        elif node_type == 'symbol' and node_data.get('symbol_type') == 'class':
            size = 18
        
        # Add node with appropriate styling
        net.add_node(
            node, 
            label=label, 
            title=title,
            color=node_colors.get(node_type, '#999999'),
            size=size,
            shape='dot' if node_type != 'directory' else 'diamond'
        )
    
    # Add edges with appropriate styles
    for source, target, edge_data in graph.edges(data=True):
        edge_type = edge_data.get('edge_type', 'unknown')
        
        # Style edges differently based on type
        if edge_type == 'import':
            color = '#f5f542'
            width = 2
            dashes = False
        elif edge_type == 'references':
            color = '#f54242'
            width = 1
            dashes = [5, 5]
        elif edge_type == 'defines':
            color = '#42f55a'
            width = 3
            dashes = False
        elif edge_type == 'contains_snippet':
            color = '#42c8f5'
            width = 1
            dashes = [2, 2]
        else:
            color = '#999999'
            width = 1
            dashes = False
        
        # Create hover title with edge details
        title = f"<div><b>{edge_type}</b><br>"
        
        if edge_data.get('line_number'):
            title += f"Line: {edge_data['line_number']}<br>"
            
        if edge_data.get('context'):
            context = edge_data['context']
            if len(context) > 200:
                context = context[:197] + "..."
            title += f"Context: {context}"
            
        title += "</div>"
        
        # Add edge with styling
        net.add_edge(
            source, 
            target, 
            title=title,
            color=color,
            width=width,
            dashes=dashes
        )
    
    # Enable physics, navigation and interaction options
    net.toggle_physics(True)
    net.show_buttons(filter_=['physics'])
    
    # Save the visualization
    net.save_graph(output_html)
    print(f"Graph visualization saved to {output_html}")
    
    return output_html

def visualize_symbol_subgraph(visualizer, symbol_name, output_html="symbol_graph.html"):
    """
    Create a focused visualization of a symbol and its relationships.
    
    Args:
        visualizer: Initialized CodebaseVisualizer instance
        symbol_name: Name of the symbol to visualize
        output_html: Output HTML file for the visualization
    """
    # Get the full graph
    full_graph = visualizer.graph
    
    # Find all nodes related to this symbol
    symbol_nodes = [node for node in full_graph.nodes() if f"::{symbol_name}" in node]
    
    if not symbol_nodes:
        print(f"Symbol '{symbol_name}' not found in the graph.")
        return None
    
    # Get nodes that are connected to symbol nodes (1-hop neighborhood)
    related_nodes = set(symbol_nodes)
    for node in symbol_nodes:
        # Add predecessors (nodes that reference this symbol)
        related_nodes.update(full_graph.predecessors(node))
        # Add successors (nodes that this symbol references)
        related_nodes.update(full_graph.successors(node))
    
    # Create a subgraph
    subgraph = full_graph.subgraph(related_nodes)
    
    # Create a pyvis network
    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    net.barnes_hut(gravity=-2000, central_gravity=0.3, spring_length=150)
    
    # Node colors by type
    node_colors = {
        'file': '#4287f5',
        'directory': '#42f5a7',
        'symbol': '#f542cb',
        'snippet': '#f5a742'
    }
    
    # Add nodes with appropriate styles
    for node, node_data in subgraph.nodes(data=True):
        node_type = node_data.get('type', 'unknown')
        label = os.path.basename(node) if '/' in node else node
        
        # Make the focus symbol nodes larger and highlighted
        if node in symbol_nodes:
            size = 30
            color = '#ff0000'  # Bright red for focus
        else:
            size = 15
            color = node_colors.get(node_type, '#999999')
        
        # Add hover information
        title = f"<div style='max-width:300px;'>"
        title += f"<b>{node}</b><br>"
        title += f"Type: {node_type}<br>"
        
        if node_type == 'symbol':
            title += f"Symbol type: {node_data.get('symbol_type', 'N/A')}<br>"
            if node_data.get('docstring'):
                title += f"Docstring: {node_data.get('docstring', '')}<br>"
                
        title += "</div>"
        
        # Add node with styling
        net.add_node(
            node,
            label=label,
            title=title,
            color=color,
            size=size
        )
    
    # Add edges with styles
    for source, target, edge_data in subgraph.edges(data=True):
        edge_type = edge_data.get('edge_type', 'unknown')
        
        # Style edges by type
        if edge_type == 'import':
            color = '#f5f542'  # Yellow
        elif edge_type == 'references':
            color = '#f54242'  # Red
        elif edge_type == 'defines':
            color = '#42f55a'  # Green
        else:
            color = '#999999'  # Gray
        
        # Add edge with details in hover
        title = f"{edge_type}"
        if edge_data.get('line_number'):
            title += f" (line {edge_data['line_number']})"
            
        net.add_edge(source, target, title=title, color=color)
    
    # Enable physics and navigation
    net.toggle_physics(True)
    net.show_buttons(filter_=['physics'])
    
    # Save the visualization
    net.save_graph(output_html)
    print(f"Symbol graph visualization saved to {output_html}")
    
    return output_html


In [ ]:
from pyvis.network import Network

def visualize_codebase_graph(codebase_path, output_html="codebase_graph.html", limit_nodes=None):
    """
    Visualize the codebase graph using pyvis.
    
    Args:
        codebase_path: Path to the codebase directory
        output_html: Output HTML file for the visualization/
        limit_nodes: Optional limit on the number of nodes to display (for large codebases)
    """
    # Initialize and build the graph
    visualizer = CodebaseVisualizer(codebase_path)
    visualizer.parse_files()
    graph = visualizer.build_graph()
    
    # Export graph to JSON (optional)
    visualizer.export_graph_json('codebase_graph.json')
    
    # Create a pyvis network
    net = Network(height="900px", width="100%", bgcolor="#222222", font_color="white")
    
    # Configure physics
    net.barnes_hut(gravity=-5000, central_gravity=0.3, spring_length=200)
    
    # Define node groups and colors
    node_colors = {
        'file': '#4287f5',
        'directory': '#42f5a7',
        'symbol': '#f542cb',
        'snippet': '#f5a742'
    }
    
    # If we need to limit nodes for performance
    if limit_nodes and len(graph.nodes()) > limit_nodes:
        # Focus on file and directory nodes, and limit symbol nodes
        important_nodes = [node for node, data in graph.nodes(data=True) 
                          if data.get('type') in ['file', 'directory']]
        
        # Add some symbol nodes to reach the limit
        symbols = [node for node, data in graph.nodes(data=True) 
                  if data.get('type') == 'symbol']
        
        # Take a subset of symbols based on connectivity
        symbol_importance = sorted(
            [(node, graph.degree(node)) for node in symbols],
            key=lambda x: x[1],
            reverse=True
        )
        
        important_symbols = [node for node, _ in symbol_importance[:limit_nodes - len(important_nodes)]]
        selected_nodes = important_nodes + important_symbols
        
        # Create a subgraph
        graph = graph.subgraph(selected_nodes)
    
    # Add nodes with appropriate styles
    for node, node_data in graph.nodes(data=True):
        node_type = node_data.get('type', 'unknown')
        label = os.path.basename(node) if '/' in node else node
        
        # Truncate very long labels
        if len(label) > 30:
            label = label[:27] + "..."
        
        # Create hover title with more details
        title = f"<div style='max-width:300px;'>"
        title += f"<b>{node}</b><br>"
        title += f"Type: {node_type}<br>"
        
        if node_type == 'file':
            title += f"Directory: {node_data.get('directory', 'N/A')}<br>"
            
        elif node_type == 'symbol':
            title += f"Symbol type: {node_data.get('symbol_type', 'N/A')}<br>"
            title += f"Line: {node_data.get('line_number', 'N/A')}<br>"
            
            if node_data.get('docstring'):
                docstring = node_data['docstring']
                if len(docstring) > 200:
                    docstring = docstring[:197] + "..."
                title += f"Docstring: {docstring}<br>"
                
        elif node_type == 'snippet':
            title += f"Lines: {node_data.get('start_line', 'N/A')}-{node_data.get('end_line', 'N/A')}<br>"
            
            if node_data.get('code_snippet'):
                snippet = node_data['code_snippet'].replace('\n', '<br>')
                if len(snippet) > 300:
                    snippet = snippet[:297] + "..."
                title += f"Code:<br><pre>{snippet}</pre>"
                
        title += "</div>"
        
        # Determine node size based on type and connections
        size = 15  # Default size
        if node_type == 'directory':
            size = 25
        elif node_type == 'file':
            size = 20
        elif node_type == 'symbol' and node_data.get('symbol_type') == 'class':
            size = 18
        
        # Add node with appropriate styling
        net.add_node(
            node, 
            label=label, 
            title=title,
            color=node_colors.get(node_type, '#999999'),
            size=size,
            shape='dot' if node_type != 'directory' else 'diamond'
        )
    
    # Add edges with appropriate styles
    for source, target, edge_data in graph.edges(data=True):
        edge_type = edge_data.get('edge_type', 'unknown')
        
        # Style edges differently based on type
        if edge_type == 'import':
            color = '#f5f542'
            width = 2
            dashes = False
        elif edge_type == 'references':
            color = '#f54242'
            width = 1
            dashes = [5, 5]
        elif edge_type == 'defines':
            color = '#42f55a'
            width = 3
            dashes = False
        elif edge_type == 'contains_snippet':
            color = '#42c8f5'
            width = 1
            dashes = [2, 2]
        else:
            color = '#999999'
            width = 1
            dashes = False
        
        # Create hover title with edge details
        title = f"<div><b>{edge_type}</b><br>"
        
        if edge_data.get('line_number'):
            title += f"Line: {edge_data['line_number']}<br>"
            
        if edge_data.get('context'):
            context = edge_data['context']
            if len(context) > 200:
                context = context[:197] + "..."
            title += f"Context: {context}"
            
        title += "</div>"
        
        # Add edge with styling
        net.add_edge(
            source, 
            target, 
            title=title,
            color=color,
            width=width,
            dashes=dashes
        )
    
    # Enable physics, navigation and interaction options
    net.toggle_physics(True)
    net.show_buttons(filter_=['physics'])
    
    # Save the visualization
    net.save_graph(output_html)
    print(f"Graph visualization saved to {output_html}")
    
    return output_html

def visualize_symbol_subgraph(visualizer, symbol_name, output_html="symbol_graph.html"):
    """
    Create a focused visualization of a symbol and its relationships.
    
    Args:
        visualizer: Initialized CodebaseVisualizer instance
        symbol_name: Name of the symbol to visualize
        output_html: Output HTML file for the visualization
    """
    # Get the full graph
    full_graph = visualizer.graph
    
    # Find all nodes related to this symbol
    symbol_nodes = [node for node in full_graph.nodes() if f"::{symbol_name}" in node]
    
    if not symbol_nodes:
        print(f"Symbol '{symbol_name}' not found in the graph.")
        return None
    
    # Get nodes that are connected to symbol nodes (1-hop neighborhood)
    related_nodes = set(symbol_nodes)
    for node in symbol_nodes:
        # Add predecessors (nodes that reference this symbol)
        related_nodes.update(full_graph.predecessors(node))
        # Add successors (nodes that this symbol references)
        related_nodes.update(full_graph.successors(node))
    
    # Create a subgraph
    subgraph = full_graph.subgraph(related_nodes)
    
    # Create a pyvis network
    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    net.barnes_hut(gravity=-2000, central_gravity=0.3, spring_length=150)
    
    # Node colors by type
    node_colors = {
        'file': '#4287f5',
        'directory': '#42f5a7',
        'symbol': '#f542cb',
        'snippet': '#f5a742'
    }
    
    # Add nodes with appropriate styles
    for node, node_data in subgraph.nodes(data=True):
        node_type = node_data.get('type', 'unknown')
        label = os.path.basename(node) if '/' in node else node
        
        # Make the focus symbol nodes larger and highlighted
        if node in symbol_nodes:
            size = 30
            color = '#ff0000'  # Bright red for focus
        else:
            size = 15
            color = node_colors.get(node_type, '#999999')
        
        # Add hover information
        title = f"<div style='max-width:300px;'>"
        title += f"<b>{node}</b><br>"
        title += f"Type: {node_type}<br>"
        
        if node_type == 'symbol':
            title += f"Symbol type: {node_data.get('symbol_type', 'N/A')}<br>"
            if node_data.get('docstring'):
                title += f"Docstring: {node_data.get('docstring', '')}<br>"
                
        title += "</div>"
        
        # Add node with styling
        net.add_node(
            node,
            label=label,
            title=title,
            color=color,
            size=size
        )
    
    # Add edges with styles
    for source, target, edge_data in subgraph.edges(data=True):
        edge_type = edge_data.get('edge_type', 'unknown')
        
        # Style edges by type
        if edge_type == 'import':
            color = '#f5f542'  # Yellow
        elif edge_type == 'references':
            color = '#f54242'  # Red
        elif edge_type == 'defines':
            color = '#42f55a'  # Green
        else:
            color = '#999999'  # Gray
        
        # Add edge with details in hover
        title = f"{edge_type}"
        if edge_data.get('line_number'):
            title += f" (line {edge_data['line_number']})"
            
        net.add_edge(source, target, title=title, color=color)
    
    # Enable physics and navigation
    net.toggle_physics(True)
    net.show_buttons(filter_=['physics'])
    
    # Save the visualization
    net.save_graph(output_html)
    print(f"Symbol graph visualization saved to {output_html}")
    
    return output_html


In [32]:
os.environ["MISTRAL_API_KEY"]="jJAuJZkjVcy2ynUhan375sHNviHiBeJU"

In [33]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    # other params...
)
llm.invoke("Hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 4, 'total_tokens': 13, 'completion_tokens': 9}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-887dd047-0991-4203-8f39-9aeae0814f00-0', usage_metadata={'input_tokens': 4, 'output_tokens': 9, 'total_tokens': 13})

In [16]:

# @tool
# def text_to_nx_algorithm_to_text(query):
#     """This tool is available to invoke a NetworkX Algorithm on
#     the ArangoDB Graph. You are responsible for accepting the
#     Natural Language Query, establishing which algorithm needs to
#     be executed, executing the algorithm, and translating the results back
#     to Natural Language, with respect to the original query.

#     If the query (e.g traversals, shortest path, etc.) can be solved using the Arango Query Language, then do not use
#     this tool.
#     """
#     llm = ChatMistralAI(
#         model="mistral-large-latest",
#         temperature=0,
#         max_retries=2,
#         # other params...
#     )
#     ######################
#     print("1) Generating NetworkX code")

#     text_to_nx = llm.invoke(f"""
#     I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

#     I have the following graph analysis query: {query}.

#     Generate the Python Code required to answer the query using the `G_adb` object.
    
#     It should give code so that is gives ALL the necessary contents that use this part of the code in terms of ALL nested imports. Consider the nested directories and sub-directory informations as well.

#     Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.

#     Only assume that networkx is installed, and other base python dependencies.

#     Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

#     Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

#     Make sure that `FINAL_RESULT` stores a short & consice answer. Avoid setting this variable to a long sequence.

#     Your code:
#     """).content

#     text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()
    
#     print('-'*10)
#     print(text_to_nx_cleaned)
#     print('-'*10)

#     ######################

#     print("\n2) Executing NetworkX code")
#     global_vars = {"G_adb": G_adb, "nx": nx}
#     local_vars = {}

#     try:
#         exec(text_to_nx_cleaned, global_vars, local_vars)
#         text_to_nx_final = text_to_nx
#     except Exception as e:
#         print(f"EXEC ERROR: {e}")
#         return f"EXEC ERROR: {e}"

#         # TODO: Consider experimenting with a code corrector!
#         attempt = 1
#         MAX_ATTEMPTS = 3

#         # while attempt <= MAX_ATTEMPTS
#             # ...

#     print('-'*10)
#     FINAL_RESULT = local_vars["FINAL_RESULT"]
#     print(f"FINAL_RESULT: {FINAL_RESULT}")
#     print('-'*10)

#     ######################

#     print("3) Formulating final answer")

#     nx_to_text = llm.invoke(f"""
#         I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

#         I have the following graph analysis query: {query}.

#         I have executed the following python code to help me answer my query:

#         ---
#         {text_to_nx_final}
#         ---

#         The `FINAL_RESULT` variable is set to the following: {FINAL_RESULT}.

#         Based on my original Query and FINAL_RESULT, generate a short and concise response to
#         answer my query.
        
#         Your response:
#     """).content

#     return nx_to_text

### Analysing the imported Graph via the prompt

In [17]:
import networkx as nx
from arango import ArangoClient
import re

def analyze_networkx_graph(G, query_text):
    """
    Process a natural language query directly against a NetworkX graph
    
    Args:
        G: NetworkX graph object
        query_text: Natural language query about the codebase
    
    Returns:
        Dictionary with analysis results in a human-readable format
    """
    # Clean the query to extract the actual symbol being sought
    clean_query = query_text.lower().strip()
    
    # Common English words to filter out
    common_words = {"what", "is", "the", "use", "of", "in", "where", "how", "why", "when", 
                    "who", "which", "does", "do", "are", "can", "could", "would", "should", 
                    "function", "method", "class", "variable", "codebase", "code", "used", 
                    "defined", "implemented", "called", "referenced"}
    
    # Find all words that could be symbols
    potential_symbols = []
    for word in clean_query.split():
        word = word.strip(".,?!()[]{}'\"\n\t")
        if len(word) > 2 and word.lower() not in common_words:
            potential_symbols.append(word)
    
    # Also look for multi-word symbols with underscores
    for i in range(len(clean_query.split()) - 1):
        compound = '_'.join(clean_query.split()[i:i+2])
        if '_' in compound and compound not in potential_symbols:
            potential_symbols.append(compound)
    
    # Extract exact symbol if passed directly
    if query_text.strip() and len(query_text.strip().split()) == 1 and '_' in query_text:
        # User likely just passed the symbol name directly
        potential_symbols = [query_text.strip()]
    
    # Find matches in the graph
    candidates = []
    
    # First, look for exact node IDs or node names
    for symbol in potential_symbols:
        for node_id, node_data in G.nodes(data=True):
            # Check if this is a symbol node
            if node_data.get('type') == 'symbol':
                # Check the node ID
                if isinstance(node_id, str) and symbol.lower() in node_id.lower():
                    candidates.append((node_id, symbol, 1.0))  # 1.0 = high confidence
                
                # Check if symbol appears in the context (code snippet)
                context = node_data.get('context', '')
                if context and symbol.lower() in context.lower():
                    # Higher confidence if it appears as a variable assignment
                    patterns = [
                        f"self.{symbol}", 
                        f"{symbol} =", 
                        f"def {symbol}", 
                        f"class {symbol}"
                    ]
                    score = 0.8  # Base score
                    for pattern in patterns:
                        if pattern.lower() in context.lower():
                            score = 0.9  # Higher confidence
                            break
                    candidates.append((node_id, symbol, score))
    
    # Second, check for symbol references in edge contexts
    if not candidates:
        for source, target, edge_data in G.edges(data=True):
            edge_type = edge_data.get('edge_type')
            if edge_type in ['references', 'defines']:
                context = edge_data.get('context', '')
                for symbol in potential_symbols:
                    if context and symbol.lower() in context.lower():
                        patterns = [
                            f"self.{symbol}", 
                            f"{symbol} =", 
                            f"def {symbol}", 
                            f"class {symbol}"
                        ]
                        score = 0.7  # Base score for edges
                        for pattern in patterns:
                            if pattern.lower() in context.lower():
                                score = 0.8  # Higher confidence
                                break
                        candidates.append((target, symbol, score))
    
    # Third, search in snippets
    if not candidates:
        for node_id, node_data in G.nodes(data=True):
            if node_data.get('type') == 'snippet':
                code_snippet = node_data.get('code_snippet', '')
                for symbol in potential_symbols:
                    if code_snippet and symbol.lower() in code_snippet.lower():
                        patterns = [
                            f"self.{symbol}", 
                            f"{symbol} =", 
                            f"def {symbol}", 
                            f"class {symbol}"
                        ]
                        score = 0.6  # Base score for snippets
                        for pattern in patterns:
                            if pattern.lower() in code_snippet.lower():
                                score = 0.7  # Higher confidence
                                break
                        # Find the file this snippet belongs to
                        file_nodes = []
                        for source, target, edge_data in G.in_edges(node_id, data=True):
                            if edge_data.get('edge_type') == 'contains_snippet':
                                file_nodes.append(source)
                        
                        # Create a pseudo symbol node ID using the file
                        if file_nodes:
                            pseudo_id = f"{file_nodes[0]}::{symbol}"
                            candidates.append((pseudo_id, symbol, score))
                        else:
                            candidates.append((node_id, symbol, score))
    
    # No matches found
    if not candidates:
        return {"response": f"Could not find a matching symbol in the codebase for '{query_text}'. Please try rephrasing your query with a specific function, class, or variable name."}
    
    # Sort candidates by confidence score
    candidates.sort(key=lambda x: x[2], reverse=True)
    
    # Choose the best candidate
    node_id, symbol_name, _ = candidates[0]
    
    # Find all usages of the identified symbol
    symbol_usages = find_symbol_usage_nx(G, node_id, symbol_name)
    
    # Generate response
    if "error" in symbol_usages:
        # Try a broader search if the specific node wasn't found
        broader_usages = search_symbol_in_contexts(G, symbol_name)
        if broader_usages:
            return {"response": broader_usages}
        return {"response": symbol_usages["error"]}
    
    # Collect information about the symbol
    definition_files = symbol_usages.get("defined_in", [])
    usage_info = symbol_usages.get("usages", {})
    symbol_type = symbol_usages.get("symbol_type", "variable")  # Default to variable
    
    # Build human-readable response
    response = f"Symbol: '{symbol_name}' (Type: {symbol_type})\n\n"
    response += f"Defined in: {', '.join(definition_files) if definition_files else 'No definition location found'}\n\n"
    
    if symbol_usages.get("docstring"):
        response += f"Documentation:\n{symbol_usages['docstring']}\n\n"
    
    response += "Used in the following locations:\n"
    
    if not usage_info:
        response += "\nNo usage information found for this symbol in graph nodes.\n"
        
        # Try a broader search in contexts
        broader_usages = search_symbol_in_contexts(G, symbol_name)
        if broader_usages:
            response += "\nHowever, found these mentions in code snippets:\n\n"
            response += broader_usages
    else:
        for file, usages in usage_info.items():
            response += f"\nFile: {file}\n"
            for usage in usages:
                line = usage.get('line', 'unknown line')
                context = usage.get('context', 'No context available')
                response += f"- Line {line}: {context}\n"
    
    # Add related symbols if available
    if symbol_usages.get("related_symbols"):
        response += "\nRelated symbols:\n"
        for related in symbol_usages["related_symbols"][:5]:  # Limit to top 5
            response += f"- {related}\n"
    
    # Add graph statistics
    response += f"\nAnalysis performed on graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges"
    
    return {"response": response}



### Searching for the symbol in both context, networkx format and file

In [18]:
def search_symbol_in_contexts(G, symbol_name):
    """
    Search for a symbol in all contexts (code snippets) in the graph
    
    Args:
        G: NetworkX graph
        symbol_name: Name of the symbol to search
    
    Returns:
        String with usage information
    """
    response = ""
    found = False
    
    # Look in snippet nodes
    for node_id, node_data in G.nodes(data=True):
        if node_data.get('type') == 'snippet':
            code_snippet = node_data.get('code_snippet', '')
            if code_snippet and symbol_name.lower() in code_snippet.lower():
                found = True
                file_name = "Unknown"
                # Find which file this snippet belongs to
                for source, target, edge_data in G.in_edges(node_id, data=True):
                    if edge_data.get('edge_type') == 'contains_snippet':
                        file_name = source
                        break
                
                start_line = node_data.get('start_line', 'unknown')
                response += f"\nFile: {file_name} (Lines {start_line}-{node_data.get('end_line', 'unknown')})\n"
                
                # Extract the lines containing the symbol
                lines = code_snippet.split('\n')
                for i, line in enumerate(lines):
                    if symbol_name.lower() in line.lower():
                        line_num = int(start_line) + i if isinstance(start_line, int) else "?"
                        response += f"- Line {line_num}: {line.strip()}\n"
    
    # Look in edge contexts
    for source, target, edge_data in G.edges(data=True):
        context = edge_data.get('context', '')
        if context and symbol_name.lower() in context.lower():
            found = True
            edge_type = edge_data.get('edge_type', 'unknown')
            line_num = edge_data.get('line_number', 'unknown')
            
            # For references or defines edges, source is usually the file
            file_name = source if edge_type in ['references', 'defines'] else "Unknown"
            
            response += f"\nFile: {file_name} (Line {line_num})\n"
            response += f"- {context.strip()}\n"
    
    if found:
        return response
    else:
        return ""

def find_symbol_usage_nx(graph, node_id, symbol_name):
    """
    Find all usages of a specific symbol across the codebase using NetworkX graph
    
    Args:
        graph: The NetworkX graph object
        node_id: The node ID of the symbol node in the graph
        symbol_name: The name of the symbol for display purposes
    
    Returns:
        Dictionary with files and line numbers where the symbol is used
    """
    # Check if this is a valid node
    if node_id not in graph:
        # This could be a pseudo node ID we created for snippet matches
        if isinstance(node_id, str) and '::' in node_id:
            file_path = node_id.split('::')[0]
            # Return information based on file path and symbol name
            return {
                "symbol": symbol_name,
                "symbol_type": "variable",  # Assuming variable as default
                "defined_in": [file_path],
                "docstring": "",
                "usages": search_symbol_in_file(graph, file_path, symbol_name),
                "related_symbols": []
            }
        return {"error": f"Symbol '{symbol_name}' not found in the codebase as a specific node"}
    
    # Get symbol node data
    node_data = graph.nodes[node_id]
    
    # Extract symbol type and other metadata
    symbol_type = node_data.get('symbol_type', 'variable')  # Default to variable
    docstring = node_data.get('docstring', '')
    
    # Find definition locations
    definitions = []
    for source, target, edge_data in graph.in_edges(node_id, data=True):
        if edge_data.get('edge_type') == 'defines':
            # Source should be a file node
            definitions.append(source)
    
    # If no definitions found through edges, extract from node ID
    if not definitions and isinstance(node_id, str) and '::' in node_id:
        file_path = node_id.split('::')[0]
        definitions.append(file_path)
    
    # Find all references to the symbol
    usages = {}
    for source, target, edge_data in graph.in_edges(node_id, data=True):
        edge_type = edge_data.get('edge_type')
        
        # Consider references
        if edge_type == 'references':
            # Source should be a file node
            file_name = source
            
            # Extract line info and context
            line_info = edge_data.get('line_number', 'unknown line')
            context = edge_data.get('context', 'No context available')
            
            if file_name not in usages:
                usages[file_name] = []
            
            usages[file_name].append({
                'line': line_info,
                'context': context
            })
    
    # Find related symbols (e.g., symbols in the same file)
    related_symbols = []
    for def_file in definitions:
        for source, target, edge_data in graph.out_edges(def_file, data=True):
            if edge_data.get('edge_type') == 'defines' and target != node_id:
                # Extract symbol name from target node ID
                if isinstance(target, str) and '::' in target:
                    related_symbol = target.split('::')[1]
                    related_symbols.append(related_symbol)
    
    return {
        "symbol": symbol_name,
        "symbol_type": symbol_type,
        "defined_in": definitions,
        "docstring": docstring,
        "usages": usages,
        "related_symbols": related_symbols
    }

def search_symbol_in_file(graph, file_path, symbol_name):
    """
    Search for uses of a symbol within a specific file
    
    Args:
        graph: NetworkX graph
        file_path: Path of the file to search in
        symbol_name: Name of the symbol to search for
    
    Returns:
        Dictionary with usage information
    """
    usages = {}
    
    # Look for snippet nodes from this file
    for node_id, node_data in graph.nodes(data=True):
        if node_data.get('type') == 'snippet':
            # Check if this snippet belongs to the file
            belongs_to_file = False
            for source, target, edge_data in graph.in_edges(node_id, data=True):
                if edge_data.get('edge_type') == 'contains_snippet' and source == file_path:
                    belongs_to_file = True
                    break
            
            if belongs_to_file:
                code_snippet = node_data.get('code_snippet', '')
                if code_snippet and symbol_name.lower() in code_snippet.lower():
                    start_line = node_data.get('start_line', 'unknown')
                    
                    if file_path not in usages:
                        usages[file_path] = []
                    
                    # Extract the lines containing the symbol
                    lines = code_snippet.split('\n')
                    for i, line in enumerate(lines):
                        if symbol_name.lower() in line.lower():
                            line_num = int(start_line) + i if isinstance(start_line, int) else "unknown"
                            usages[file_path].append({
                                'line': line_num,
                                'context': line.strip()
                            })
    
    return usages

### The text LLM file

In [19]:
def convert_arango_to_networkx(db, graph_name):
    """
    Convert an ArangoDB graph to NetworkX format
    
    Args:
        db: ArangoDB database connection
        graph_name: Name of the graph in ArangoDB
    
    Returns:
        NetworkX graph object
    """
    G = nx.DiGraph()
    
    try:
        # Get graph from ArangoDB
        arango_graph = db.graph(graph_name)
        
        # Get graph properties
        try:
            graph_properties = arango_graph.properties()
            edge_definitions = graph_properties.get('edgeDefinitions', [])
            
            # Process each vertex collection
            vertex_collections = set()
            for edge_def in edge_definitions:
                vertex_collections.update(edge_def.get('from', []))
                vertex_collections.update(edge_def.get('to', []))
                
            # Add nodes from vertex collections
            for collection_name in vertex_collections:
                try:
                    collection = db.collection(collection_name)
                    for doc in collection:
                        node_id = doc['_id']
                        G.add_node(node_id, **doc)
                except Exception as e:
                    print(f"Error with vertex collection {collection_name}: {str(e)}")
            
            # Process edge collections
            for edge_def in edge_definitions:
                edge_collection_name = edge_def.get('collection')
                if edge_collection_name:
                    try:
                        collection = db.collection(edge_collection_name)
                        for doc in collection:
                            from_id = doc['_from']
                            to_id = doc['_to']
                            G.add_edge(from_id, to_id, **doc)
                    except Exception as e:
                        print(f"Error with edge collection {edge_collection_name}: {str(e)}")
                        
        except Exception as e:
            # Alternative approach: try to infer collections from naming convention
            print(f"Error getting graph properties, trying alternate approach: {str(e)}")
            
            # Common naming patterns for ArangoDB collections
            node_collection_pattern = re.compile(f"{re.escape(graph_name)}_node")
            edge_collection_pattern = re.compile(f"{re.escape(graph_name)}_.*_to_.*")
            
            # Try to find matching collections
            for collection_name in db.collections():
                if node_collection_pattern.match(collection_name):
                    try:
                        collection = db.collection(collection_name)
                        for doc in collection:
                            node_id = doc['_id']
                            G.add_node(node_id, **doc)
                    except Exception as e:
                        print(f"Error with inferred vertex collection {collection_name}: {str(e)}")
                        
                elif edge_collection_pattern.match(collection_name):
                    try:
                        collection = db.collection(collection_name)
                        for doc in collection:
                            from_id = doc['_from']
                            to_id = doc['_to']
                            G.add_edge(from_id, to_id, **doc)
                    except Exception as e:
                        print(f"Error with inferred edge collection {collection_name}: {str(e)}")
            
    except Exception as e:
        print(f"Error accessing ArangoDB graph: {str(e)}")
    
    return G

In [20]:
def text_to_nx_algorithm_to_text(query_text, graph=None, db_connection=None, graph_name=None):
    """
    Universal entry point for processing natural language queries against a code graph
    
    Args:
        query_text: Natural language query about the codebase
        graph: NetworkX graph object (if already available)
        db_connection: ArangoDB connection (if graph should be loaded from ArangoDB)
        graph_name: Name of the graph in ArangoDB (if db_connection is provided)
    
    Returns:
        Dictionary with analysis results in a human-readable format
    """
    # Case 1: NetworkX graph is directly provided
    if graph is not None:
        if isinstance(graph, nx.Graph):
            return analyze_networkx_graph(graph, query_text)
        else:
            return {"response": "Provided graph object is not a valid NetworkX graph"}
    
    # Case 2: ArangoDB connection is provided but no graph
    elif db_connection is not None and graph_name is None:
        try:
            # Try to list available graphs
            try:
                available_graphs = db_connection.graphs()
                if available_graphs:
                    graph_name = available_graphs[0]["name"]
                else:
                    return {"response": "No graphs found in the ArangoDB database"}
            except:
                # Different API for networkx-arangodb
                try:
                    import networkx_arangodb as nxadb
                    # Try a different approach for networkx-arangodb
                    available_graphs = [g for g in db_connection.graphs()]
                    if available_graphs:
                        graph_name = available_graphs[0]
                    else:
                        return {"response": "No graphs found in the ArangoDB database"}
                except:
                    return {"response": "Could not retrieve graph list from ArangoDB"}
        except Exception as e:
            return {"response": f"Error retrieving graphs from ArangoDB: {str(e)}"}
        
    
    # Case 3: ArangoDB connection and graph name are provided
    if db_connection is not None and graph_name is not None:
        try:
            # Try to load the graph using standard ArangoDB driver
            try:
                # Check if graph_name is a string
                if not isinstance(graph_name, str):
                    if hasattr(graph_name, 'name'):
                        # It might be a graph object with a name attribute
                        graph_name = graph_name.name
                    else:
                        return {"response": "Graph name must be a string or an object with a name attribute"}
                
                # Try to get the graph from ArangoDB
                arango_graph = db_connection.graph(graph_name)
                # Convert to NetworkX using our custom function
                nx_graph = convert_arango_to_networkx(db_connection, graph_name)
                
                return analyze_networkx_graph(nx_graph, query_text)
            except Exception as e1:
                # If standard approach fails, try networkx-arangodb
                try:
                    import networkx_arangodb as nxadb
                    graph = nxadb.Graph(name=graph_name, db=db_connection)
                    nx_graph = graph.to_networkx()
                    print(type(nx_graph))
                    return analyze_networkx_graph(nx_graph, query_text)
                except Exception as e2:
                    return {"response": f"Error loading graph from ArangoDB: {str(e1)}\nAlternative method error: {str(e2)}"}
        except Exception as e:
            return {"response": f"Error accessing ArangoDB: {str(e)}"}
    
    # Case 4: No graph info provided at all
    return {"response": "Please provide either a NetworkX graph or ArangoDB connection details"}



### Converting it to NETWORKX, can be removed for effiency and maintained in arango

In [21]:
type(G_adb)

nx_arangodb.classes.graph.Graph

In [22]:
import networkx as nx

# Create an empty NetworkX DiGraph
G = nx.DiGraph()

# Check what methods and attributes are available on your G_adb object
# Uncomment this line to inspect what's available:
print(dir(G_adb))

# Try accessing nodes and edges directly
# Option 1: If G_adb has nodes() and edges() methods
try:
    # Add nodes
    for node_id, node_data in G_adb.nodes(data=True):
        G.add_node(node_id, **node_data)
    
    # Add edges
    for source, target, edge_data in G_adb.edges(data=True):
        G.add_edge(source, target, **edge_data)
except AttributeError:
    # Option 2: If G_adb has direct node and edge collections
    try:
        # Check if G_adb.nodes and G_adb.edges are iterable
        for node_id, node_data in G_adb.nodes.items():
            G.add_node(node_id, **node_data)
        
        for edge_id, edge_data in G_adb.edges.items():
            # Assuming edge_data has 'from' and 'to' attributes
            G.add_edge(edge_data['from'], edge_data['to'], **edge_data)
    except AttributeError:
        print("Could not determine how to access nodes and edges. Please inspect G_adb structure.")

['_Graph__db', '_Graph__graph_exists_in_db', '_Graph__is_smart', '_Graph__name', '_Graph__set_arangodb_backend_config', '_Graph__set_db', '_Graph__set_edge_collections_attributes', '_Graph__set_graph', '_Graph__smart_field', '_Graph__use_arango_views', '__annotations__', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__networkx_backend__', '__networkx_cache__', '__networkx_plugin__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adj', '_db_name', '_edge_collections_attributes', '_hosts', '_load_nx_graph', '_loaded_incoming_graph_data', '_node', '_password', '_set_factory_methods', '_username', 'adb_graph', 'add_edge', 'add_edges_from', 'add_node', 'add_node_override', 'add_nodes_from

In [28]:
result = text_to_nx_algorithm_to_text(
    "has_level_handler()",
    graph=G,  # Your NetworkX graph
    #graph_name="FlaskRepv1",
    db_connection=db
)
print(result['response'])

Symbol: 'has_level_handler()' (Type: function)

Defined in: No definition location found

Used in the following locations:

No usage information found for this symbol in graph nodes.

However, found these mentions in code snippets:


File: Unknown (Lines 61-80)
- Line 70: def test_has_level_handler():

File: FlaskRepv1_node/213 (Line 67)
- assert wsgi_errors_stream._get_current_object() is sys.stderr

    with app.test_request_context(errors_stream=stream):
        assert wsgi_errors_stream._get_current_object() is stream


def test_has_level_handler():

Analysis performed on graph with 2975 nodes and 8651 edges


In [ ]:
print(G_adb)

In [1]:
arango_graph.schema

NameError: name 'arango_graph' is not defined